# **Projeto de Previsão de Inadimplência**

---
## **Motivação:**
A inadimplência é um problema crítico para instituições financeiras, uma vez que impacta diretamente o fluxo de caixa e a saúde financeira das empresas. Prever com antecedência quais clientes têm maior chance de se tornarem inadimplentes permite que as empresas adotem estratégias preventivas, minimizando prejuízos.



---
## **Objetivos:**

Este projeto tem como **principal objetivo** construir um modelo de Machine Learning que seja capaz de prever a inadimplência de clientes com base em um conjunto de dados de contratos de financiamento.

- **Construir um modelo preditivo de inadimplência** utilizando algoritmos de classificação.
- **Analisar as variáveis mais relevantes** para prever a inadimplência, ajudando a entender os fatores de risco.
- **Avaliar a performance dos modelos** utilizando métricas de performance.


---

#### Entre as colunas deste dataset temos:
| Coluna                        | Descrição                                          |
|-------------------------------|----------------------------------------------------|
| **NUMERO_CONTRATO**            | Número identificador do contrato                   |
| **DATA_ASSINATURA_CONTRATO**   | Data em que o contrato foi assinado                |
| **TIPO_FINANCIAMENTO**         | Tipo de financiamento oferecido                    |
| **TAXA_AO_ANO**                | Taxa de juros anual do contrato                    |
| **PZ_FINANCIAMENTO**           | Prazo de financiamento (em meses)                  |
| **CIDADE_CLIENTE**             | Cidade onde o cliente reside                       |
| **ESTADO_CLIENTE**             | Estado onde o cliente reside                       |
| **RENDA_MENSAL_CLIENTE**       | Renda mensal do cliente                            |
| **QT_PC_ATRASO**               | Quantidade de parcelas em atraso                   |
| **QT_DIAS_PRIM_PC_ATRASO**     | Quantidade de dias em atraso da primeira parcela   |
| **QT_TOTAL_PC_PAGAS**          | Quantidade total de parcelas pagas                 |
| **VL_TOTAL_PC_PAGAS**          | Valor total das parcelas pagas                     |
| **QT_PC_PAGAS_EM_DIA**         | Quantidade de parcelas pagas em dia                |
| **QT_DIAS_MIN_ATRASO**         | Menor quantidade de dias de atraso                 |
| **QT_DIAS_MAX_ATRASO**         | Maior quantidade de dias de atraso                 |
| **QT_DIAS_MEDIA_ATRASO**       | Média de dias em atraso                            |
| **VALOR_FINANCIAMENTO**        | Valor total do financiamento                       |
| **VALOR_PARCELA**              | Valor de cada parcela                              |
| **IDADE_DATA_ASSINATURA_CONTRATO** | Idade do cliente no momento da assinatura do contrato |
| **INADIMPLENTE_COBRANCA**      | Indicador se o cliente está inadimplente           |



**A coluna alvo é "INADIMPLENTE_COBRANCA"**

#### Este é um problema de **Classificação**

---

## **Etapas do projeto**

1. **Carregamento e análise inicial dos dados**: Vamos carregar o dataset e verificar suas características iniciais.

2. **Exploração e tratamento dos dados**: Identificar dados ausentes, remover outliers (se necessário), e preparar as variáveis para modelagem.

3. **Análise Estatística e Engenharia de Atributos**: Para entender as características e padrões dos dados e melhorar a qualidade dos dados.

4. **Visualização com Gráficos**: Visualizar os dados de uma forma gráfica para ajudar na compreensão.

5. **Treinamento do modelo**: Construir e treinar modelos preditivos usando algoritmos de Machine Learning.

6. **Avaliação do modelo**: Avaliar a performance dos modelos com base nas métricas adequadas.

7. **Colocar o modelo em producão**: Para que as previsões e insights gerados pelo modelo possam ser utilizados em cenários do mundo real.

8. **Análise Prescritiva**: Conjunto de possíveis soluções para diminuir a quantidade de clientes inadimplentes.

---

### Conectando com o Banco de dados e Importando os dados

In [1]:
#importando as bibliotecas
import warnings
import time
import pandas as pd
import numpy as np
import pypyodbc as sql
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import joblib
from dotenv import load_dotenv
from sklearn.preprocessing import LabelEncoder, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, accuracy_score, recall_score, precision_score
from xgboost.sklearn import XGBClassifier
from imblearn.over_sampling import SMOTE

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:.2f}".format
warnings.filterwarnings("ignore")

In [2]:
#carregando as variaveis de ambiente
load_dotenv()

USUARIO = os.environ.get("USUARIO")
SENHA = os.environ.get("SENHA")
HOST = os.environ.get("HOST")
DATABASE = os.environ.get("DATABASE")


In [3]:
#importar os dados do SQL

connection_string = "Driver={SQL Server};Server=" + HOST + ";Database=" + DATABASE + ";UID=" + USUARIO + ";PWD=" + SENHA + ";"

conexao = sql.connect(connection_string)

df_original = pd.read_sql_query("SELECT * FROM EXTRACAO_DADOS_SISTEMA", conexao)

conexao.close()

### Começando a Análise Exploratória

In [4]:
#observando o numero de linhas e colunas
print(f"O dataset contém {df_original.shape[0]} linhas e {df_original.shape[1]} colunas.")

O dataset contém 10415 linhas e 20 colunas.


In [5]:
#observando todas as colunas e os tipos das mesmas, depois, iremos otimizar este dataframe
df_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   numero_contrato                 10415 non-null  int64  
 1   data_assinatura_contrato        10415 non-null  object 
 2   tipo_financiamento              10415 non-null  object 
 3   taxa_ao_ano                     10415 non-null  float64
 4   pz_financiamento                10415 non-null  int64  
 5   cidade_cliente                  10415 non-null  object 
 6   estado_cliente                  10415 non-null  object 
 7   renda_mensal_cliente            10415 non-null  float64
 8   qt_pc_atraso                    10415 non-null  int64  
 9   qt_dias_prim_pc_atraso          10415 non-null  int64  
 10  qt_total_pc_pagas               10415 non-null  int64  
 11  vl_total_pc_pagas               10411 non-null  float64
 12  qt_pc_paga_em_dia               

In [6]:
#visao geral dos dados
df_original.sample(10)

,numero_contrato,data_assinatura_contrato,tipo_financiamento,taxa_ao_ano,pz_financiamento,cidade_cliente,estado_cliente,renda_mensal_cliente,qt_pc_atraso,qt_dias_prim_pc_atraso,qt_total_pc_pagas,vl_total_pc_pagas,qt_pc_paga_em_dia,qt_dias_min_atraso,qt_dias_max_atraso,qt_dias_media_atraso,valor_financiamento,valor_parcela,idade_data_assinatura_contrato,inadimplente_cobranca
7970,56026,2015-07-14,IMOBILIARIO,18.00,180,LUCAS DO RIO VERDE,MT,5800.00,81,2639,1,581.83,1,0,0,0,88748.00,581.79,65.00,NAO
8118,79237,2016-07-12,IMOBILIARIO,20.00,72,BRASILIA,DF,7800.00,60,2215,3,1438.72,3,0,0,0,210000.00,3000.00,45.00,NAO
9669,80200,2017-01-24,IMOBILIARIO,17.00,72,SAO PAULO,SP,9800.00,0,0,9,5026.58,9,0,0,0,245000.00,3485.42,33.00,SIM
5086,76635,2013-02-08,IMOBILIARIO,16.00,72,CALDAZINHA,GO,7800.00,0,0,60,51773.96,24,3,264,72,264138.00,3752.44,29.00,SIM
3889,72591,2015-08-14,IMOBILIARIO,20.00,240,BRASILIA,DF,7800.00,0,0,88,130708.09,70,1,50,11,350000.00,1750.00,35.00,SIM
2815,132073,2014-01-24,IMOBILIARIO,17.00,100,SAO PAULO,SP,1800.00,59,2841,12,11385.66,7,13,20,16,547589.00,5608.88,44.00,NAO
6937,77244,2015-12-11,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,0,0,43,28975.21,41,2,5,3,265482.00,3776.80,68.00,SIM
2398,78373,2015-05-15,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,0,0,61,31556.44,60,87,87,87,245000.00,3485.42,44.00,NAO
5413,79261,2016-09-13,IMOBILIARIO,20.00,72,GOIANIA,GO,7800.00,0,0,31,29030.55,17,1,99,16,280000.00,4000.00,54.00,SIM
5364,55219,2018-04-20,IMOBILIARIO,22.00,220,BRASILIA,DF,5800.00,0,0,56,64697.35,54,13,13,13,275000.00,1525.00,40.00,SIM


In [7]:
#vendo metricas das colunas numericas
df_original.describe()

,numero_contrato,taxa_ao_ano,pz_financiamento,renda_mensal_cliente,qt_pc_atraso,qt_dias_prim_pc_atraso,qt_total_pc_pagas,vl_total_pc_pagas,qt_pc_paga_em_dia,qt_dias_min_atraso,qt_dias_max_atraso,qt_dias_media_atraso,valor_financiamento,valor_parcela,idade_data_assinatura_contrato
count,10415.00,10415.00,10415.00,10415.00,10415.00,10415.00,10415.00,10411.00,10415.00,10415.00,10415.00,10415.00,10415.00,10415.00,10414.00
mean,95569.36,17.27,114.24,5620.26,16.72,664.39,35.95,44528.55,27.67,6.12,79.13,28.84,325590.86,3617.34,40.79
std,34561.66,2.57,64.33,2930.30,30.71,1181.35,28.38,65640.97,25.15,82.91,334.87,137.40,177477.49,1932.71,12.39
min,32709.00,7.00,48.00,1800.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,29327.00,185.74,0.00
25%,75868.50,16.00,72.00,1800.00,0.00,0.00,11.00,8675.44,7.00,0.00,0.00,0.00,210000.00,2467.76,31.25
50%,79111.00,17.00,72.00,7800.00,0.00,0.00,33.00,24345.60,22.00,1.00,8.00,4.00,280000.00,3470.59,39.00
75%,136637.50,19.00,180.00,7800.00,22.00,1005.50,54.00,46787.29,42.00,3.00,62.00,22.00,392973.77,4930.25,49.00
max,155890.00,25.50,240.00,9800.00,178.00,5655.00,167.00,714499.74,136.00,5465.00,5710.00,5480.00,1400000.00,14410.00,118.00


- Aqui observamos alguns potenciais outliers como idade_data_assinatura_contrato minima: 0 e idade_data_assinatura_contrato maxima: 118
---

In [8]:
#vendo metricas das colunas nao numericas
df_original.describe(exclude="number")

,data_assinatura_contrato,tipo_financiamento,cidade_cliente,estado_cliente,inadimplente_cobranca
count,10415,10415,10415,10415,10415
unique,1070,1,525,27,2
top,2014-06-13,IMOBILIARIO,BRASILIA,DF,SIM
freq,187,10415,3737,4240,8038


In [9]:
#filtro para ver a quantidade media de parcelas em atraso por renda mensal do cliente
filtro_atraso = df_original.groupby("renda_mensal_cliente")["qt_pc_atraso"].mean()
print(filtro_atraso)

maior_id_atraso = filtro_atraso.idxmax()
maior_media_atraso = filtro_atraso.max()

print(f"\nA renda_mensal_cliente com a maior média de qt_pc_atraso é R$ {maior_id_atraso} com uma média de {maior_media_atraso:.2f} de parcelas em atraso")

renda_mensal_cliente
1800.00    5.60
3800.00    0.06
4800.00    8.12
5800.00   31.69
6800.00   26.58
7800.00   22.82
9800.00   16.73
Name: qt_pc_atraso, dtype: float64

A renda_mensal_cliente com a maior média de qt_pc_atraso é R$ 5800.0 com uma média de 31.69 de parcelas em atraso


- A quantidade média de parcelas atrasadas é bem maior quando o cliente tem uma renda mensal de mais de **R$ 5.800.**

---

In [10]:
#filtro para ver a idade media do cliente na data de assinatura do contrato por renda mensal do cliente
df_original.groupby("renda_mensal_cliente")["idade_data_assinatura_contrato"].mean()

renda_mensal_cliente
1800.00   40.98
3800.00   45.00
4800.00   42.25
5800.00   41.90
6800.00   44.77
7800.00   40.37
9800.00   39.48
Name: idade_data_assinatura_contrato, dtype: float64

- A média de idade na hora da assinatura do contrato é bem parecida em todos os níveis de renda com a média total.

---

In [11]:
#filtro para ver a quantidade media do valor das parcelas por renda mensal do cliente
filtro_renda = df_original.groupby("renda_mensal_cliente")["valor_parcela"].mean()
print(filtro_renda)

maior_id_venda = filtro_renda.idxmax()
maior_media_venda = filtro_renda.max()

print(f"\nA renda_mensal_cliente com a maior média de valor_parcela é R$ {maior_id_venda} com uma média de R$ {maior_media_venda:.2f} por parcela")



renda_mensal_cliente
1800.00   4733.81
3800.00   1143.67
4800.00   1040.42
5800.00   1265.53
6800.00   2843.32
7800.00   3498.73
9800.00   3910.99
Name: valor_parcela, dtype: float64

A renda_mensal_cliente com a maior média de valor_parcela é R$ 1800.0 com uma média de R$ 4733.81 por parcela


- A média do valor da parcela por renda mensal do cliente é bem maior quando o cliente recebe **R$ 1.800 ou menos**, ou, quando o cliente recebe **R$ 6.800 ou mais**.

---

In [12]:
#observando o inicio e fim dos dados da coluna data_assinatura_contrato
inicio = pd.to_datetime(df_original["data_assinatura_contrato"]).dt.date.min() 
fim = pd.to_datetime(df_original["data_assinatura_contrato"]).dt.date.max()

print("Período dos dados de:", inicio, "até", fim)

Período dos dados de: 2012-01-10 até 2022-11-14


- Aqui observamos que só existe um tipo de financiamento nestes dados: **IMOBILIARIO**
- Clientes de **525 cidades diferentes**
- Clientes de **todos** os estados do Brasil
- Maior parte dos clientes neste conjunto de dados está **inadimplente (8038 de 10415, ~77%)**
- Período dos dados de: **2012-01-10 até 2022-11-14**


---

In [13]:
#filtro para ver a porcentagem de valores por renda mensal do cliente
print(df_original["renda_mensal_cliente"].value_counts(normalize=True))
print(f"\nA maior parte dos clientes ({df_original["renda_mensal_cliente"].value_counts().max()}), recebe entre R$ 6.900 e R$ {df_original["renda_mensal_cliente"].value_counts().idxmax()}")

renda_mensal_cliente
7800.00   0.41
1800.00   0.34
5800.00   0.10
9800.00   0.09
4800.00   0.04
6800.00   0.01
3800.00   0.00
Name: proportion, dtype: float64

A maior parte dos clientes (4306), recebe entre R$ 6.900 e R$ 7800.0


- A maior parte dos clientes (**41%**) recebe entre **R$ 6.900 e R$ 7.800 mensais**
- **34%** dos clientes recebem até **R$ 1.800**
- **75%** dos cliente ou recebem entre **R$ 6.900 e R$ 7.800 ou até R$ 1.800**
          
---

In [14]:
#visualizando a quantidade de linhas onde a idade na data de assinatura é maior ou menor que um valor
total = len(df_original.query("idade_data_assinatura_contrato < 10"))
total2 = len(df_original.query("idade_data_assinatura_contrato < 20"))
total3 = len(df_original.query("idade_data_assinatura_contrato > 50"))
total4 = len(df_original.query("idade_data_assinatura_contrato > 70"))
print(f"Número de linhas onde a idade_data_assinatura_contrato é menor que 10: {total}")
print(f"Número de linhas onde a idade_data_assinatura_contrato é menor que 20: {total2}")
print(f"Número de linhas onde a idade_data_assinatura_contrato é maior que 50: {total3}")
print(f"Número de linhas onde a idade_data_assinatura_contrato é maior que 70: {total4}")


Número de linhas onde a idade_data_assinatura_contrato é menor que 10: 7
Número de linhas onde a idade_data_assinatura_contrato é menor que 20: 80
Número de linhas onde a idade_data_assinatura_contrato é maior que 50: 2303
Número de linhas onde a idade_data_assinatura_contrato é maior que 70: 175


- Existem **7** registros onde a idade na data de assinatura é menor que 10 anos, um possível erro 
- **80** clientes com menos de **20 anos** na data de assinatura
- **2303** clientes com mais de **50 anos** na data de assinatura
- **175** clientes com mais de **70 anos** na data de assinatura
          
---

In [15]:
#porcentagem de parcelas pagas em dia e em atraso

total_pagas = df_original["qt_total_pc_pagas"].sum()
pagas_em_dia = df_original["qt_pc_paga_em_dia"].sum()

porcentagem_em_dia = (pagas_em_dia / total_pagas) * 100
porcentagem_atraso = 100 - porcentagem_em_dia

print(f"Porcentagem de parcelas pagas em dia: {porcentagem_em_dia:.2f}%")
print(f"Porcentagem de parcelas pagas em atraso: {porcentagem_atraso:.2f}%")


Porcentagem de parcelas pagas em dia: 76.97%
Porcentagem de parcelas pagas em atraso: 23.03%


- **76%** das parcelas pagas, são pagas em dia
---

In [16]:
#verificando a quantidade de valores únicos de cada coluna
df_original.nunique()

numero_contrato                   10415
data_assinatura_contrato           1070
tipo_financiamento                    1
taxa_ao_ano                         111
pz_financiamento                      9
cidade_cliente                      525
estado_cliente                       27
renda_mensal_cliente                  7
qt_pc_atraso                        130
qt_dias_prim_pc_atraso              227
qt_total_pc_pagas                   137
vl_total_pc_pagas                  9860
qt_pc_paga_em_dia                   129
qt_dias_min_atraso                  122
qt_dias_max_atraso                  534
qt_dias_media_atraso                323
valor_financiamento                 834
valor_parcela                      1826
idade_data_assinatura_contrato       78
inadimplente_cobranca                 2
dtype: int64

- Verificamos novamente que **tipo_financiamento** possuí apenas um tipo de dado
- **vl_total_pc_pagas** possuí alta cardinalidade 
- Apenas 7 tipos diferentes de **renda_mensal_cliente**                
---

Faremos mais análises na parte de **Visualização com gráficos**


### Otimização do dataframe

In [17]:
#otimizando a coluna Data_Contratacao para datetime64[ns]                  
df_original["data_assinatura_contrato"] = pd.to_datetime(df_original["data_assinatura_contrato"])

In [18]:
#otimizando colunas categoricas
colunas_categoricas = ["tipo_financiamento", "cidade_cliente", "estado_cliente", "inadimplente_cobranca"]

for coluna in colunas_categoricas:
	df_original[coluna] = df_original[coluna].astype("category")


In [19]:
#otimizando colunas numericas
colunas_float = df_original.select_dtypes(include="float64").columns
colunas_int = df_original.select_dtypes(include="int64").columns

df_original[colunas_float] = df_original[colunas_float].apply(pd.to_numeric, downcast="float")
df_original[colunas_int] = df_original[colunas_int].apply(pd.to_numeric, downcast="integer")


In [20]:
#verificando a mudança de tamanho do dataset
df_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   numero_contrato                 10415 non-null  int32         
 1   data_assinatura_contrato        10415 non-null  datetime64[ns]
 2   tipo_financiamento              10415 non-null  category      
 3   taxa_ao_ano                     10415 non-null  float32       
 4   pz_financiamento                10415 non-null  int16         
 5   cidade_cliente                  10415 non-null  category      
 6   estado_cliente                  10415 non-null  category      
 7   renda_mensal_cliente            10415 non-null  float32       
 8   qt_pc_atraso                    10415 non-null  int16         
 9   qt_dias_prim_pc_atraso          10415 non-null  int16         
 10  qt_total_pc_pagas               10415 non-null  int16         
 11  vl

- O tamanho do dataframe diminuiu de ***1.6+ MB para 683.0 KB**
---

### Tratamento dos dados

In [21]:
#removendo linhas duplicadas 
df_original.drop_duplicates()
print(f"O dataset contém {df_original.shape[0]} linhas e {df_original.shape[1]} colunas.")

O dataset contém 10415 linhas e 20 colunas.


- Não foram encontradas linhas duplicadas
---

In [22]:
#verificando valores nulos
print(f"O número de valores vazios de cada coluna \n\n{df_original.isnull().sum()}")

O número de valores vazios de cada coluna 

numero_contrato                   0
data_assinatura_contrato          0
tipo_financiamento                0
taxa_ao_ano                       0
pz_financiamento                  0
cidade_cliente                    0
estado_cliente                    0
renda_mensal_cliente              0
qt_pc_atraso                      0
qt_dias_prim_pc_atraso            0
qt_total_pc_pagas                 0
vl_total_pc_pagas                 4
qt_pc_paga_em_dia                 0
qt_dias_min_atraso                0
qt_dias_max_atraso                0
qt_dias_media_atraso              0
valor_financiamento               0
valor_parcela                     0
idade_data_assinatura_contrato    1
inadimplente_cobranca             0
dtype: int64


- Identificamos que ***vl_total_pc_pagas**  e **idade_data_assinatura_contrato** possuem 4 e 1 valor vazio respectivamente, iremos excluir essas linhas já que são poucas e não irão afetar o modelo
---

In [23]:
#excluindo linhas com valores nulos
df_original.dropna(inplace=True)

In [24]:
#verificando novamente se existem valores nulos
print(f"O número de valores vazios de cada coluna \n\n{df_original.isnull().sum()}")


O número de valores vazios de cada coluna 

numero_contrato                   0
data_assinatura_contrato          0
tipo_financiamento                0
taxa_ao_ano                       0
pz_financiamento                  0
cidade_cliente                    0
estado_cliente                    0
renda_mensal_cliente              0
qt_pc_atraso                      0
qt_dias_prim_pc_atraso            0
qt_total_pc_pagas                 0
vl_total_pc_pagas                 0
qt_pc_paga_em_dia                 0
qt_dias_min_atraso                0
qt_dias_max_atraso                0
qt_dias_media_atraso              0
valor_financiamento               0
valor_parcela                     0
idade_data_assinatura_contrato    0
inadimplente_cobranca             0
dtype: int64


- Todos os valores nulos foram excluídos
---

In [25]:
#verificando a possibilidade de outliers
df_original[df_original["idade_data_assinatura_contrato"] <= 17]

,numero_contrato,data_assinatura_contrato,tipo_financiamento,taxa_ao_ano,pz_financiamento,cidade_cliente,estado_cliente,renda_mensal_cliente,qt_pc_atraso,qt_dias_prim_pc_atraso,qt_total_pc_pagas,vl_total_pc_pagas,qt_pc_paga_em_dia,qt_dias_min_atraso,qt_dias_max_atraso,qt_dias_media_atraso,valor_financiamento,valor_parcela,idade_data_assinatura_contrato,inadimplente_cobranca
856,79171,2017-11-14,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,0,0,35,35294.51,10,1,55,30,412755.00,5871.93,9.00,SIM
1418,79545,2016-11-11,IMOBILIARIO,20.00,72,BRASILIA,DF,7800.00,0,0,38,33545.05,3,1,245,61,420000.00,6000.00,8.00,SIM
2044,137914,2019-04-09,IMOBILIARIO,14.65,72,PLANALTINA DF,DF,1800.00,0,0,44,31670.84,42,2,2,2,385000.00,5459.13,12.00,SIM
2219,79450,2016-04-12,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,0,0,53,22320.79,41,1,82,21,210000.00,2987.50,17.00,SIM
2253,76537,2012-02-10,IMOBILIARIO,16.00,72,BRASILIA,DF,7800.00,0,0,56,28877.66,44,11,178,61,175000.00,2486.11,16.00,SIM
2979,76481,2012-02-10,IMOBILIARIO,16.00,72,BRASILIA,DF,7800.00,0,0,63,35871.16,44,8,68,34,175000.00,2486.11,16.00,SIM
3002,136455,2018-12-11,IMOBILIARIO,14.65,72,SAO PAULO,SP,1800.00,0,0,43,27952.17,37,2,21,5,280000.00,3970.28,0.00,SIM
3142,75101,2012-01-10,IMOBILIARIO,15.00,72,PLANALTINA,GO,7800.00,59,3769,6,5106.58,4,20,49,34,373590.00,5299.94,17.00,NAO
3562,136456,2018-12-11,IMOBILIARIO,14.65,72,SAO PAULO,SP,1800.00,0,0,43,27955.99,37,2,21,8,280000.00,3970.28,0.00,SIM
3749,78832,2016-02-16,IMOBILIARIO,17.00,72,ALTAMIRA,PA,7800.00,0,0,51,34921.82,11,1,154,43,210000.00,2987.50,7.00,SIM


- Alguns destes valores aparentam ser outliers, mas como são poucos, eles serão tratados na parte de Normalização/ Padronização dos dados
---

In [26]:
#eliminando colunas inúteis para a nossa análise

df_original = df_original.drop(["numero_contrato", "data_assinatura_contrato", "tipo_financiamento"], axis=1)


- Eliminando **numero_contrato** pois não tem relevância para nossa análise
- Eliminando **data_assinatura_contrato** pois não se trata de uma série temporal
- Eliminando **tipo_financiamento** pois todas as linhas possuem o mesmo valor

---

In [27]:
#transformando nossas colunas para caixa alta
df_original.columns = df_original.columns.str.upper()

In [28]:
#renomeando algumas colunas
df_original.rename(columns={"VALOR_FINANCIAMENTO": "VL_FINANCIAMENTO", "VALOR_PARCELA": "VL_PARCELA",  }, inplace=True)

In [29]:
#visualizando novamente as colunas
df_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   TAXA_AO_ANO                     10410 non-null  float32 
 1   PZ_FINANCIAMENTO                10410 non-null  int16   
 2   CIDADE_CLIENTE                  10410 non-null  category
 3   ESTADO_CLIENTE                  10410 non-null  category
 4   RENDA_MENSAL_CLIENTE            10410 non-null  float32 
 5   QT_PC_ATRASO                    10410 non-null  int16   
 6   QT_DIAS_PRIM_PC_ATRASO          10410 non-null  int16   
 7   QT_TOTAL_PC_PAGAS               10410 non-null  int16   
 8   VL_TOTAL_PC_PAGAS               10410 non-null  float64 
 9   QT_PC_PAGA_EM_DIA               10410 non-null  int16   
 10  QT_DIAS_MIN_ATRASO              10410 non-null  int16   
 11  QT_DIAS_MAX_ATRASO              10410 non-null  int16   
 12  QT_DIAS_MEDIA_ATRASO   

## Engenharia de Atributos

In [30]:
#verificando a cardinalidade das colunas
df_original.nunique()

TAXA_AO_ANO                        111
PZ_FINANCIAMENTO                     9
CIDADE_CLIENTE                     525
ESTADO_CLIENTE                      27
RENDA_MENSAL_CLIENTE                 7
QT_PC_ATRASO                       130
QT_DIAS_PRIM_PC_ATRASO             227
QT_TOTAL_PC_PAGAS                  136
VL_TOTAL_PC_PAGAS                 9859
QT_PC_PAGA_EM_DIA                  129
QT_DIAS_MIN_ATRASO                 122
QT_DIAS_MAX_ATRASO                 534
QT_DIAS_MEDIA_ATRASO               323
VL_FINANCIAMENTO                   834
VL_PARCELA                        1826
IDADE_DATA_ASSINATURA_CONTRATO      78
INADIMPLENTE_COBRANCA                2
dtype: int64

In [31]:
#verificando valores minimos e maximos para algumas colunas 

df_original[["VL_TOTAL_PC_PAGAS", "VL_FINANCIAMENTO", "VL_PARCELA", "IDADE_DATA_ASSINATURA_CONTRATO"]].describe().loc[["min", "mean", "max", "25%", "50%", "75%"]]


,VL_TOTAL_PC_PAGAS,VL_FINANCIAMENTO,VL_PARCELA,IDADE_DATA_ASSINATURA_CONTRATO
min,0.01,29327.00,185.74,0.00
mean,44530.61,325570.97,3617.72,40.79
max,714499.74,1400000.00,14410.00,118.00
25%,8674.22,210000.00,2471.04,31.25
50%,24345.61,280000.00,3470.59,39.00
75%,46807.04,392973.77,4931.86,49.00


In [32]:
#criando faixa de valores
faixas = [-100, 8_675, 25_000, 47_000, 9_999_999]
categorias = ["Até R$ 8.675", "de R$ 8.676 até R$ 25.000", "de R$ 25.001 até R$ 47.000", "Mais de R$ 47.000"]
df_original["FAIXA_VL_TOTAL_PC_PAGAS"] = pd.cut(df_original["VL_TOTAL_PC_PAGAS"], bins=faixas, labels=categorias)
df_original["FAIXA_VL_TOTAL_PC_PAGAS"] = df_original["FAIXA_VL_TOTAL_PC_PAGAS"].cat.set_categories(categorias, ordered = True)
print(f"É ordenada? : {df_original["FAIXA_VL_TOTAL_PC_PAGAS"].cat.ordered}")
print(df_original["FAIXA_VL_TOTAL_PC_PAGAS"].unique())


faixas = [-100, 210_000, 290_000, 400_000, 9_999_999]
categorias = ["Até R$ 210.000", "de R$ 210.000 até R$ 290.000", "de R$ 290.001 até R$ 400.000", "Mais de R$ 400.000"]
df_original["FAIXA_VALOR_FINANCIAMENTO"] = pd.cut(df_original["VL_FINANCIAMENTO"], bins=faixas, labels=categorias)
df_original["FAIXA_VALOR_FINANCIAMENTO"] = df_original["FAIXA_VALOR_FINANCIAMENTO"].cat.set_categories(categorias, ordered = True)
print("\n")
print(f"É ordenada? : {df_original["FAIXA_VALOR_FINANCIAMENTO"].cat.ordered}")
print(df_original["FAIXA_VALOR_FINANCIAMENTO"].unique())

faixas = [-100, 2_500, 3_500, 5_000, 999999]
categorias = ["Até R$ 2.500", "de R$ 2.501 até R$ 3.500", "de R$ 3.501 até R$ 5.000", "Mais de R$ 5.000"]
df_original["FAIXA_VALOR_PARCELA"] = pd.cut(df_original["VL_PARCELA"], bins=faixas, labels=categorias)
print("\n")
print(f"É ordenada? : {df_original["FAIXA_VALOR_PARCELA"].cat.ordered}")
print(df_original["FAIXA_VALOR_PARCELA"].unique())

faixas = [-100, 18, 25, 35, 45, 55, 65, 999]
categorias = ["0-17", "18-24", "25-34", "35-44", "45-54", "55-64", "65+"]
df_original["FAIXA_IDADE_DATA_ASSINATURA_CONTRATO"] = pd.cut(df_original["IDADE_DATA_ASSINATURA_CONTRATO"], bins=faixas, labels=categorias)
print("\n")
print(f"É ordenada? : {df_original["FAIXA_IDADE_DATA_ASSINATURA_CONTRATO"].cat.ordered}")
print(df_original["FAIXA_IDADE_DATA_ASSINATURA_CONTRATO"].unique())

É ordenada? : True
['Mais de R$ 47.000', 'de R$ 25.001 até R$ 47.000', 'de R$ 8.676 até R$ 25.000', 'Até R$ 8.675']
Categories (4, object): ['Até R$ 8.675' < 'de R$ 8.676 até R$ 25.000' < 'de R$ 25.001 até R$ 47.000' < 'Mais de R$ 47.000']


É ordenada? : True
['Até R$ 210.000', 'de R$ 290.001 até R$ 400.000', 'Mais de R$ 400.000', 'de R$ 210.000 até R$ 290.000']
Categories (4, object): ['Até R$ 210.000' < 'de R$ 210.000 até R$ 290.000' < 'de R$ 290.001 até R$ 400.000' < 'Mais de R$ 400.000']


É ordenada? : True
['Até R$ 2.500', 'de R$ 2.501 até R$ 3.500', 'de R$ 3.501 até R$ 5.000', 'Mais de R$ 5.000']
Categories (4, object): ['Até R$ 2.500' < 'de R$ 2.501 até R$ 3.500' < 'de R$ 3.501 até R$ 5.000' < 'Mais de R$ 5.000']


É ordenada? : True
['65+', '25-34', '55-64', '35-44', '45-54', '18-24', '0-17']
Categories (7, object): ['0-17' < '18-24' < '25-34' < '35-44' < '45-54' < '55-64' < '65+']


- Criamos faixas de valores em algumas colunas e as ordenamos para diminuirmos a cardinalidade delas
---

In [33]:
#verificando os intervalos
print(df_original.groupby(["FAIXA_VL_TOTAL_PC_PAGAS"]).size())
print("\n")
print(df_original.groupby(["FAIXA_VALOR_FINANCIAMENTO"]).size())
print("\n")
print(df_original.groupby(["FAIXA_VALOR_PARCELA"]).size())
print("\n")
print(df_original.groupby(["FAIXA_IDADE_DATA_ASSINATURA_CONTRATO"]).size())

FAIXA_VL_TOTAL_PC_PAGAS
Até R$ 8.675                  2603
de R$ 8.676 até R$ 25.000     2702
de R$ 25.001 até R$ 47.000    2512
Mais de R$ 47.000             2593
dtype: int64


FAIXA_VALOR_FINANCIAMENTO
Até R$ 210.000                  3411
de R$ 210.000 até R$ 290.000    2350
de R$ 290.001 até R$ 400.000    2276
Mais de R$ 400.000              2373
dtype: int64


FAIXA_VALOR_PARCELA
Até R$ 2.500                3233
de R$ 2.501 até R$ 3.500    2722
de R$ 3.501 até R$ 5.000    2603
Mais de R$ 5.000            1852
dtype: int64


FAIXA_IDADE_DATA_ASSINATURA_CONTRATO
0-17       43
18-24     731
25-34    3288
35-44    3127
45-54    1761
55-64    1025
65+       435
dtype: int64


In [34]:
#excluindo as colunas antigas
colunas = ["CIDADE_CLIENTE", "ESTADO_CLIENTE", "RENDA_MENSAL_CLIENTE", "FAIXA_IDADE_DATA_ASSINATURA_CONTRATO", "TAXA_AO_ANO", "PZ_FINANCIAMENTO", "QT_PC_ATRASO", 
           "QT_DIAS_PRIM_PC_ATRASO", "QT_TOTAL_PC_PAGAS", "QT_PC_PAGA_EM_DIA", "QT_DIAS_MIN_ATRASO", "QT_DIAS_MEDIA_ATRASO", "QT_DIAS_MAX_ATRASO", 
           "FAIXA_VL_TOTAL_PC_PAGAS", "FAIXA_VALOR_FINANCIAMENTO", "FAIXA_VALOR_PARCELA", "INADIMPLENTE_COBRANCA"]

df_tratado = pd.DataFrame(df_original, columns = colunas)

In [35]:
#olhando o dataset para verificarmos a exclusao das colunas antigas e criacao das novas colunas
df_tratado.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   CIDADE_CLIENTE                        10410 non-null  category
 1   ESTADO_CLIENTE                        10410 non-null  category
 2   RENDA_MENSAL_CLIENTE                  10410 non-null  float32 
 3   FAIXA_IDADE_DATA_ASSINATURA_CONTRATO  10410 non-null  category
 4   TAXA_AO_ANO                           10410 non-null  float32 
 5   PZ_FINANCIAMENTO                      10410 non-null  int16   
 6   QT_PC_ATRASO                          10410 non-null  int16   
 7   QT_DIAS_PRIM_PC_ATRASO                10410 non-null  int16   
 8   QT_TOTAL_PC_PAGAS                     10410 non-null  int16   
 9   QT_PC_PAGA_EM_DIA                     10410 non-null  int16   
 10  QT_DIAS_MIN_ATRASO                    10410 non-null  int16   
 11  QT_DIAS

In [36]:
#visualizando os dados
df_tratado.sample(10)

,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,FAIXA_IDADE_DATA_ASSINATURA_CONTRATO,TAXA_AO_ANO,PZ_FINANCIAMENTO,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MEDIA_ATRASO,QT_DIAS_MAX_ATRASO,FAIXA_VL_TOTAL_PC_PAGAS,FAIXA_VALOR_FINANCIAMENTO,FAIXA_VALOR_PARCELA,INADIMPLENTE_COBRANCA
6452,SAO PAULO,SP,1800.00,45-54,12.55,72,0,0,13,10,1,1,1,de R$ 8.676 até R$ 25.000,Mais de R$ 400.000,Mais de R$ 5.000,SIM
1962,MANAUS,AM,7800.00,25-34,17.00,72,0,0,47,45,1,2,3,de R$ 25.001 até R$ 47.000,de R$ 210.000 até R$ 290.000,de R$ 2.501 até R$ 3.500,SIM
8754,BELO HORIZONTE,MG,7800.00,35-44,17.00,72,70,2519,2,2,0,0,0,Até R$ 8.675,de R$ 210.000 até R$ 290.000,de R$ 3.501 até R$ 5.000,NAO
6767,SAO PAULO,SP,1800.00,55-64,14.65,72,0,0,13,12,1,1,1,de R$ 8.676 até R$ 25.000,Mais de R$ 400.000,Mais de R$ 5.000,SIM
7688,FORMOSA,GO,7800.00,25-34,17.00,72,0,0,48,48,0,0,0,de R$ 25.001 até R$ 47.000,de R$ 210.000 até R$ 290.000,de R$ 3.501 até R$ 5.000,SIM
4629,NOVA VENEZA,GO,1800.00,25-34,17.50,72,0,0,34,14,1,10,28,de R$ 8.676 até R$ 25.000,Até R$ 210.000,de R$ 2.501 até R$ 3.500,SIM
5240,FORMOSA,GO,5800.00,55-64,19.00,220,53,1595,4,3,6,6,6,Até R$ 8.675,de R$ 290.001 até R$ 400.000,Até R$ 2.500,SIM
3315,BRASILIA,DF,7800.00,35-44,19.00,72,48,1441,7,2,2,8,17,Até R$ 8.675,de R$ 210.000 até R$ 290.000,de R$ 3.501 até R$ 5.000,NAO
6450,MARABA,PA,1800.00,35-44,16.05,220,0,0,20,17,2,15,28,de R$ 25.001 até R$ 47.000,de R$ 210.000 até R$ 290.000,Até R$ 2.500,SIM
4173,CANDANGOLANDIA,DF,1800.00,18-24,18.50,72,4,110,14,5,1,49,105,Até R$ 8.675,de R$ 210.000 até R$ 290.000,de R$ 3.501 até R$ 5.000,SIM


## Análise Estatística

In [37]:
#verificando novamente as métricas para cada coluna
df_tratado.describe() 

,RENDA_MENSAL_CLIENTE,TAXA_AO_ANO,PZ_FINANCIAMENTO,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MEDIA_ATRASO,QT_DIAS_MAX_ATRASO
count,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00
mean,5619.98,17.27,114.21,16.69,663.71,35.96,27.68,6.12,28.85,79.16
std,2930.53,2.57,64.31,30.68,1180.94,28.38,25.15,82.93,137.43,334.95
min,1800.00,7.00,48.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,1800.00,16.00,72.00,0.00,0.00,11.00,7.00,0.00,0.00,0.00
50%,7800.00,17.00,72.00,0.00,0.00,33.00,22.00,1.00,4.00,8.00
75%,7800.00,19.00,180.00,22.00,990.00,54.00,42.00,3.00,22.00,62.00
max,9800.00,25.50,240.00,178.00,5655.00,167.00,136.00,5465.00,5480.00,5710.00


In [38]:
df_tratado.describe(exclude="number")

,CIDADE_CLIENTE,ESTADO_CLIENTE,FAIXA_IDADE_DATA_ASSINATURA_CONTRATO,FAIXA_VL_TOTAL_PC_PAGAS,FAIXA_VALOR_FINANCIAMENTO,FAIXA_VALOR_PARCELA,INADIMPLENTE_COBRANCA
count,10410,10410,10410,10410,10410,10410,10410
unique,525,27,7,4,4,4,2
top,BRASILIA,DF,25-34,de R$ 8.676 até R$ 25.000,Até R$ 210.000,Até R$ 2.500,SIM
freq,3734,4237,3288,2702,3411,3233,8035


In [39]:
#verificando a distribuicao de valores da coluna PZ_Financiamento
print(df_tratado.groupby(["PZ_FINANCIAMENTO"]).size())

PZ_FINANCIAMENTO
48      343
72     6121
100     938
160       2
180     765
194       2
200     558
220     773
240     908
dtype: int64


- A mediana da **Taxa ao Ano** é de **17.27%**.
- Média de **Prazo de Financiamento** é de **114** enquanto a mediana é de **72**. O pulo de 25% dos dados para 50% dos dados não existe, mas de 50% para 75% sai de **72** para **180**. O desvio padrão também é alto, o que indica uma maior dispersão dos dados.
- Algo semelhante occore com as demais colunas do Dataset, onde a maior parte dos valores está concentrada em uma pequena faixa de valores.
- Maior parte dos clientes é de **Brasília**.
- A faixa do **Valor Total de Parcelas Pagas** que mais aparece é de **R$ 8.676 até R$ 25.000**.
- A faixa do **Valor de Financiamento** que mais aparece é de **Até R$ 210.000**.
- A faixa do **Valor da Parcela** que mais aparece é de **Até R$ 2.500**.	
---

## Visualização com Gráficos

In [40]:
# #grafico de barras mostrando a Distribuição dos Clientes por Renda

# df_count = df_tratado.groupby("RENDA_MENSAL_CLIENTE").size().reset_index(name="Count")

# df_count["RENDA_MENSAL_CLIENTE"] = df_count["RENDA_MENSAL_CLIENTE"]

# total_clientes = df_count["Count"].sum()
# df_count["Porcento"] = (df_count["Count"] / total_clientes) * 100

# #ordenando os dados
# df_count = df_count.sort_values(by="RENDA_MENSAL_CLIENTE", ascending=True)

# fig = px.bar(df_count, x="RENDA_MENSAL_CLIENTE", y="Count", text_auto=True, 
#              template="plotly_dark", 
#              text=df_count["Porcento"],
#              color="Count", color_continuous_scale = "Purples")

# fig.update_traces(dict(marker_line_width=0),
#                    hovertemplate="<b>Renda Mensal: %{x}</b><br>Número de Clientes: %{y}", #Mudando o que aparece quando se coloca o mouse em cima das barras
#                    texttemplate="%{y} Clientes<br>%{text:.2f}%" #mostrando as porcentagens do número total de clientes
# )

# df_count["Cumulativo"] = df_count["Count"].cumsum()
# fig.add_trace(go.Scatter(x=df_count["RENDA_MENSAL_CLIENTE"], #linha amarela que mostra o total de clientes
#                          y=df_count["Cumulativo"], 
#                          mode="lines", 
#                          name="Total", 
#                          line=dict(color="#cc7722", width=1)))


# fig.update_layout(
#     title_text="Distribuição dos Clientes por Renda", #titulo do grafico
#     title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#     title_x=0.5, #titulo centralizado
#     font=dict(size=14, color="darkgray"), #fonte
#     xaxis_title="Renda Mensal (R$)", #titulo eixo X
#     xaxis_type="category", #tipo do eixo X
#     xaxis_tickformat=",.0f", #formatacao do eixo X
#     xaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo x
#     yaxis_title="Número de Clientes", #titulo do eixo Y
#     yaxis_tickformat=",.0f", #formatacao do eixo Y
#     yaxis_showgrid=True, #mostrando as linhas no eixo Y
#     yaxis_gridwidth=1, #espessura das linhas no eixo Y
#     yaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo Y
#     coloraxis_colorbar=dict(x=1.1, #ajusta a posicao do label da barra de cores
#     title="Número de Clientes", #muda o titulo da legenda da barra de cores                 
#     )
# )

# fig.show()

# fig.write_html("html/RENDA_MENSAL_CLIENTE.html", full_html=False, include_plotlyjs="cdn") #convertendo para html
# pio.write_image(fig, "imagens/RENDA_MENSAL_CLIENTE.png", width=2677, height=450 , scale=1) #convertendo para png

- A maior concentração de clientes está na faixa de **R$ 7.800* seguido por **R$ 1.800**
---

In [41]:
# media_salario_idade = df_tratado.groupby("FAIXA_IDADE_DATA_ASSINATURA_CONTRATO")["RENDA_MENSAL_CLIENTE"].mean().reset_index() #achando a media de salario por idade

# media_salario = df_tratado["RENDA_MENSAL_CLIENTE"].mean() #achando a media de salario geral

# top_2 = media_salario_idade.nlargest(2, "RENDA_MENSAL_CLIENTE") #achando os 2 maiores valores da media de salario por idade

# media_salario_idade["Cor"] = media_salario_idade["RENDA_MENSAL_CLIENTE"].apply(   # pintando os 2 maiores valores de roxo, e os demais de cinza claro
#     lambda x: "purple" if x in top_2["RENDA_MENSAL_CLIENTE"].values else "darkgray"
# )

# fig = px.bar(media_salario_idade, x="FAIXA_IDADE_DATA_ASSINATURA_CONTRATO", y="RENDA_MENSAL_CLIENTE", text_auto=True, 
#              color="Cor", color_discrete_sequence=["purple", "darkgray"] ,
#              template="plotly_dark",
#              labels={
#                  "RENDA_MENSAL_CLIENTE": "Renda Mensal Média (R$)",
#                  "FAIXA_IDADE_DATA_ASSINATURA_CONTRATO": "Faixa Etária"
#              },
#              )

# fig.add_shape(type="line", #linha para a media de salario geral
#               x0=-0.5,  
#               y0=media_salario, 
#               x1=len(media_salario_idade) - 0.5,
#               y1=media_salario,
#               line=dict(color="#cc7722", width=3))


# fig.add_annotation(    #anotacao para a media de salario geral
#     x=len(media_salario_idade) - 1,
#     y=media_salario,
#     text=f"Media: R$ {media_salario:.2f}",
#     showarrow=True,
#     arrowhead=1,
#     arrowcolor="#cc7722",
#     ax=50,
#     ay=60,
#     font=dict(size=16, color="#cc7722")
# )

# fig.update_layout(
#     title_text="Renda Mensal Média por Faixa Etária",
#     title_font=dict(size=30, color="#cc7722"),
#     title_x=0.5,
#     font=dict(size=14, color="darkgray"),
#     xaxis_title="Faixa Etária (Anos)",
#     xaxis_title_font=dict(size=20, color="#cc7722"),
#     yaxis_title="Renda Mensal Média (R$)",
#     yaxis_title_font=dict(size=20, color="#cc7722"),
#     yaxis_tickprefix="R$",
# )

# fig.update_traces(textposition="outside")

# fig.for_each_trace(lambda t: t.update(name="2 Maiores Valores" if "purple" in t.marker.color else "Menores Valores")) #mudando a legenda do lado direito

# fig.show()

# fig.write_html("html/RENDA_MENSAL_CLIENTE_FAIXA_ETARIA.html", full_html=False, include_plotlyjs="cdn") #convertendo para html
# pio.write_image(fig, "imagens/RENDA_MENSAL_CLIENTE_FAIXA_ETARIA.png", width=2677, height=450 , scale=1) #convertendo para png

- A maior média de Renda mensal está entre **0-24 anos**
---

In [42]:
# atraso_medio_financiamento = df_tratado.groupby("FAIXA_VALOR_FINANCIAMENTO").agg({"QT_DIAS_MEDIA_ATRASO": "mean"}).reset_index() #pega a media de dias de atraso da coluna QT_DIAS_MEDIA_ATRASO
# atraso_medio = df_tratado["QT_DIAS_MEDIA_ATRASO"].mean()

# fig = px.bar(atraso_medio_financiamento, x="FAIXA_VALOR_FINANCIAMENTO", y="QT_DIAS_MEDIA_ATRASO", text_auto=True,
#              template="plotly_dark",
#              color="QT_DIAS_MEDIA_ATRASO", 
#              color_continuous_scale="Purples",
#              labels={"QT_DIAS_MEDIA_ATRASO": "Atraso Médio (Dias)"})

# fig.add_shape(type="line", #linha para a media de dias de atraso
#               x0=-0.5,  
#               y0=atraso_medio, 
#               x1=len(atraso_medio_financiamento) - 0.5,
#               y1=atraso_medio,
#               line=dict(color="#cc7722", width=3))

# fig.add_annotation(    #anotacao para a media de salario geral
#     x=len(atraso_medio_financiamento) - 1,
#     y=atraso_medio,
#     text=f"Media: {atraso_medio:.2f} Dias",
#     showarrow=True,
#     arrowhead=1,
#     arrowcolor="#cc7722",
#     ax=-50,
#     ay=-60,
#     font=dict(size=16, color="#cc7722")
# )

# fig.update_layout(
#     title_text="Atraso Médio por Faixa de Valor do Financiamento", #titulo do grafico
#     title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#     title_x=0.5, #titulo centralizado
#     font=dict(size=14, color="darkgray"), #fonte
#     xaxis_title="Faixa de Valor do Financiamento", #titulo eixo X
#     xaxis_title_font=dict(size=20, color="#cc7722"),
#     yaxis_title="Atraso Médio (Dias)",
#     yaxis_tickformat=",.0f", #formatacao do eixo X #titulo do eixo Y
#     yaxis_showgrid=True, #mostrando as linhas no eixo Y
#     yaxis_gridwidth=1, #espessura das linhas no eixo Y
#     yaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo Y
#     coloraxis_colorbar=dict(x=1.1)         
# )

# fig.update_traces(hovertemplate="<b>Renda Mensal: %{x}</b><br>Média do Número de Clientes: %{y}") #Mudando o que aparece quando se coloca o mouse em cima das barras

# fig.show()

# fig.write_html("html/FAIXA_VALOR_FINANCIAMENTO_QT_DIAS_MEDIA_ATRASO.html", full_html=False, include_plotlyjs="cdn") #convertendo para html
# pio.write_image(fig, "imagens/FAIXA_VALOR_FINANCIAMENTO_QT_DIAS_MEDIA_ATRASO.png", width=2677, height=450, scale=1) #convertendo para png

- Aqui podemos verificar qual a Faixa de Valor do Financiamento tendem a ter a maior média de atraso, neste caso, a faixa de até R$ 210.000 possui a maior média.
---


In [43]:
# df_tratado = df_tratado.sort_values(by="FAIXA_IDADE_DATA_ASSINATURA_CONTRATO", ascending=True)

# fig = go.Figure()

# fig.add_trace(go.Violin(x=df_tratado["FAIXA_IDADE_DATA_ASSINATURA_CONTRATO"], y=df_tratado["PZ_FINANCIAMENTO"], 
#                  line_color="#cc7722",
#                  line_width=0.5,
#                  fillcolor="purple")
# )

# fig.update_traces(dict(marker_line_width=0))

# fig.update_layout(
#     title_text="Relação entre Idade na Data de Assinatura do Contrato e Prazo de Financiamento", #titulo do grafico
#     title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#     title_x=0.5, #titulo centralizado
#     font=dict(size=14, color="darkgray"), #fonte
#     xaxis_title="Idade na Assinatura do Contrato (Anos)", #titulo eixo X
#     xaxis_title_font=dict(size=20, color="#cc7722"),
#     xaxis_tickformat=",.0f", #formatacao do eixo X
#     xaxis_type="category",
#     yaxis_title="Prazo de Financiamento (Meses)",
#     yaxis_tickformat=",.0f", #formatacao do eixo X #titulo do eixo Y
#     yaxis_showgrid=True, #mostrando as linhas no eixo Y
#     yaxis_gridwidth=1, #espessura das linhas no eixo Y
#     yaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo Y
#     coloraxis_colorbar=dict(x=1.1),
#     template="plotly_dark",
# )

# fig.show()

# fig.write_html("html/FAIXA_IDADE_DATA_ASSINATURA_CONTRATO_PZ_FINANCIAMENTO.html", full_html=False, include_plotlyjs="cdn") #convertendo para html
# pio.write_image(fig, "imagens/FAIXA_IDADE_DATA_ASSINATURA_CONTRATO_PZ_FINANCIAMENTO.png", width=2677, height=450, scale=1) #convertendo para png

- Entre **0-24 anos** há uma densidade visível ao redor de prazos de 50 a 100 meses, sugerindo que, embora alguns tenham prazos longos, a maioria dos clientes se concentra em prazos médios de financiamento.

- Os clientes com **mais de 25 anos** tem uma densidade concentrada entre 50 e 100 meses, com muito menos variabilidade em comparação com as faixas de **0-24 anos**.

- Na faixa de **25 anos ou mais** há menos outliers, o que indica menos casos extremos de financiamentos muito longos ou curtos.

- Para o grupo de **65+**, a distribuição do prazo de financiamento é ainda mais estreita e centrada, o que pode indicar que clientes mais velhos têm opções de financiamento mais previsíveis ou são mais cautelosos ao escolher prazos mais curtos.

- À medida que a idade aumenta, os prazos de financiamento se tornam mais padronizados e centrados entre **50 e 100 meses**.

---

In [44]:
# df_tratado = df_tratado.sort_values(by="RENDA_MENSAL_CLIENTE", ascending=True)

# fig = go.Figure()

# fig.add_trace(go.Violin(x=df_tratado["RENDA_MENSAL_CLIENTE"], y=df_tratado["QT_DIAS_MAX_ATRASO"], 
#                 line_color="#cc7722",
#                 line_width=0.5,
#                 fillcolor="purple")
# )

# fig.update_traces(dict(marker_line_width=0))
# fig.update_traces(hovertemplate="<b>Renda Mensal: %{x}</b><br>Dias de Atraso Máximo: %{y}") #Mudando o que aparece quando se coloca o mouse em cima das barras

# fig.update_layout(
#     title_text="Distribuição de Atrasos Máximos por Renda Mensal", #titulo do grafico
#     title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#     title_x=0.5, #titulo centralizado
#     font=dict(size=14, color="darkgray"), #fonte
#     xaxis_title="Renda Mensal (R$)", #titulo eixo X
#     xaxis_title_font=dict(size=20, color="#cc7722"),
#     xaxis_tickformat=",.0f", #formatacao do eixo X
#     xaxis_type="category",
#     yaxis_title="Atraso Máximo (Dias)",
#     yaxis_tickformat=",.0f", #formatacao do eixo X #titulo do eixo Y
#     yaxis_showgrid=True, #mostrando as linhas no eixo Y
#     yaxis_gridwidth=1, #espessura das linhas no eixo Y
#     yaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo Y
#     coloraxis_colorbar=dict(x=1.1),
#     template="plotly_dark",
# )


# fig.show()

# fig.write_html("html/RENDA_MENSAL_CLIENTE_QT_DIAS_MAX_ATRASO.html", full_html=False, include_plotlyjs="cdn") #convertendo para html
# pio.write_image(fig, "imagens/RENDA_MENSAL_CLIENTE_QT_DIAS_MAX_ATRASO.png", width=2677, height=450, scale=1) #convertendo para png

- Para a maioria das faixas de renda, existe uma grande variação dos atrasos. Os atrasos variam de **0 até mais de 3000 dias**.

- A maioria dos clientes em todas as faixas de renda tendem a ter atrasos menores, próxima a **0 dias**. Isso significa que, embora alguns clientes tenham atrasos longos, a maioria consegue manter atrasos mais curtos.

- Algumas faixas de renda apresentam caudas mais longas, alcançando atrasos extremos de mais de **5000 dias**.

- Existem alguns outliers espalhados por todas as faixas de renda, com alguns casos extremos de atrasos muito além do que a maioria dos clientes experimenta.

---

In [45]:
# df_tratado = df_tratado.sort_values(by="FAIXA_VALOR_FINANCIAMENTO", ascending=True)


# fig = go.Figure()

# fig.add_trace(go.Box(x=df_tratado["FAIXA_VALOR_FINANCIAMENTO"], y=df_tratado["FAIXA_VALOR_PARCELA"],
#             boxpoints="all",
#             marker_color = "purple"))


# fig.update_traces(dict(marker_line_width=0))

# fig.update_layout(
#     title_text="Valor da Parcela vs Valor do Financiamento", #titulo do grafico
#     title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#     title_x=0.5, #titulo centralizado
#     font=dict(size=14, color="darkgray"), #fonte
#     xaxis_title="Valor do Financiamento (R$)", #titulo eixo X
#     xaxis_title_font=dict(size=20, color="#cc7722"),
#     xaxis_tickformat=",.0f", #formatacao do eixo X
#     xaxis_type="category",
#     yaxis_title= "Valor da Parcela (R$)",
#     yaxis_tickformat=",.0f", #formatacao do eixo X #titulo do eixo Y
#     yaxis_showgrid=True, #mostrando as linhas no eixo Y
#     yaxis_gridwidth=1, #espessura das linhas no eixo Y
#     yaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo Y
#     coloraxis_colorbar=dict(x=1.1),
#     template="plotly_dark",
# )


# fig.show()

# fig.write_html("html/FAIXA_VALOR_FINANCIAMENTO_FAIXA_VALOR_PARCELA.html", full_html=False, include_plotlyjs="cdn") #convertendo para html
# pio.write_image(fig, "imagens/FAIXA_VALOR_FINANCIAMENTO_FAIXA_VALOR_PARCELA.png", width=2677, height=450, scale=1) #convertendo para png


- Os Financiamentos **Até R$ 210.000** faixa estão concentrados em valores de parcela mais baixos **(menos de R$ 2.500)**
- Os Financiamentos **de R$ 210.000 até R$ 290.000** mostram uma maior variabilidade nos valores das parcelas, com uma caixa e *whiskers* mais amplos, indicando uma dispersão maior nos valores das prestações.
- Os Financiamentos **de R$ 290.000 até R$ 400.000** possuem a maior parte dos valores de parcelas **até R$ 5.000**, com alguns passando desta faixa.
- Os Financiamentos **Acima de R$ 400.000** O gráfico mostra ainda mais variabilidade nas parcelas, com valores medianos mais altos. Também há alguns outliers, mas a faixa de pagamentos aqui se estende para valores de parcelas mais altos.
- Conforme o **valor do financiamento aumenta** , as **parcelas mensais tendem a aumentar**, e a dispersão dos valores também se torna maior.
- A média da **Taxa de Juros** aumenta conforme o **Valor do Financiamento**.

---    

In [46]:
# soma_qt_pc_atraso = df_tratado["QT_PC_ATRASO"].sum()
# soma_qt_total_pc_pagas = df_tratado["QT_TOTAL_PC_PAGAS"].sum()

# fig = go.Figure()

# fig.add_trace(go.Pie(
#     labels=["Total de Parcelas em Atraso", "Total de Parcelas Pagas"],
#     values=[soma_qt_pc_atraso, soma_qt_total_pc_pagas],  
#     hole=0.5, 
#     marker=dict(colors=["purple", "gray"], pattern=dict(shape=["", "-"])), 
#     hovertemplate=(
#         "%{label}: %{value}<br>" +
#         "Porcentagem: %{percent}<extra></extra>"
#     ), 
#     textinfo="label+percent", 
#     textposition="outside",
#     pull=[0.1, 0]  
# ))

# fig.update_layout(
#     title_text="Proporção entre Soma de Parcelas em Atraso e Soma de Parcelas Pagas", #titulo do grafico
#     title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#     title_x=0.5, #titulo centralizado
#     font=dict(size=14, color="#cc7722"), #fonte
#     template="plotly_dark",
# )

# fig.show()

# fig.write_html("html/QT_PC_ATRASO_QT_TOTAL_PC_PAGAS.html", full_html=False, include_plotlyjs="cdn")
# fig.write_image("imagens/QT_PC_ATRASO_QT_TOTAL_PC_PAGAS.png", width=2677, height=450, scale=1)

- **31.7%** das parcelas estão em atraso, sendo um total de **173,754 parcelas**.
- Este número parece bastante elevado se comparado com dados de **2021 no Brasil, no financiamento imobiliário**, que chegou a **20%**.

**FONTE:**

**Fundos Ampliam Compra de Crédito Imobiliário em Atraso no Brasil**

https://www.infomoney.com.br/onde-investir/fundos-ampliam-compra-de-credito-imobiliario-em-atraso-no-brasil/

Data de acesso: 22/10/2024


---

In [47]:
# df_tratado = df_tratado.sort_values(by="PZ_FINANCIAMENTO", ascending=True)

# media_taxa = df_tratado.groupby("PZ_FINANCIAMENTO").agg({"TAXA_AO_ANO": "mean"}).reset_index()

# fig = px.bar(media_taxa, 
#                 x="PZ_FINANCIAMENTO", 
#                 y="TAXA_AO_ANO",
#                 color = "TAXA_AO_ANO",
#                 color_continuous_scale = "Purples"
# )

                                
# fig.update_traces(dict(marker_line_width=0),
#                   hovertemplate="<b>Prazo de Financiamento: %{x} meses</b><br>Média da Taxa ao Ano: %{y}%" #Mudando o que aparece quando se coloca o mouse em cima das barras
# )

# fig.update_layout(
#     title_text="Prazo de Financiamento x Taxa ao Ano", #titulo do grafico
#     title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#     title_x=0.5, #titulo centralizado
#     font=dict(size=14, color="darkgray"), #fonte
#     xaxis_title="Prazo de Financiamento (meses)", #titulo eixo X
#     xaxis_title_font=dict(size=20, color="#cc7722"),
#     xaxis_tickformat=",.0f", #formatacao do eixo X
#     xaxis_type="category",
#     yaxis_title="Média da Taxa ao Ano (%)",
#     yaxis_tickformat=",.0f", #formatacao do eixo X #titulo do eixo Y
#     yaxis_showgrid=True, #mostrando as linhas no eixo Y
#     yaxis_gridwidth=1, #espessura das linhas no eixo Y
#     yaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo Y
#     coloraxis_colorbar=dict(x=1.1,title="Média da Taxa ao Ano (%)"),
#     template="plotly_dark",
# )

# fig.show()

# fig.write_html("html/PZ_FINANCIAMENTO_TAXA_AO_ANO.html", full_html=False, include_plotlyjs="cdn")
# fig.write_image("imagens/PZ_FINANCIAMENTO_TAXA_AO_ANO.png", width=2677, height=450, scale=1)

- Para o prazo de Financiamento de **48 meses**, a taxa média de juros é a mais alta, por volta de **20% ao ano**.

- Nos prazos **entre 160 e 194 meses**, a taxa de juros média diminui um pouco, ficando em torno de **14-17%** ao ano. Esses prazos intermediários parecem ser os mais favoráveis em termos de taxa de juros.

- Prazos mais longos, **entre 200 e 240 meses**, as taxas voltam a subir, chegando a **20%**.

- Prazos mais curtos ou muito longos estão associados a taxas de juros mais altas, enquanto prazos intermediários oferecem as melhores condições.

---

In [48]:
# df_count = df_tratado["ESTADO_CLIENTE"].value_counts().reset_index()
# df_count.columns = ["ESTADO_CLIENTE", "Quantidade"]

# fig = px.choropleth(df_count, 
#                     geojson="https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson", 
#                     locations="ESTADO_CLIENTE", 
#                     featureidkey="properties.sigla", 
#                     color="Quantidade",
#                     title="Clientes por Estados",
#                     scope="south america")

# fig.update_geos(fitbounds="locations", visible=False)

# fig.update_layout(
#     title_text="Número de clientes por estado",
#     title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#     title_x=0.5, #titulo centralizado
#     font=dict(size=14, color="darkgray"),
#     margin={"r":0,"t":50,"l":0,"b":0}, 
#     coloraxis_colorbar=dict(x=1.1,title="Total de Clientes por Estado"),
#     template="plotly_dark",
# )


# fig.show()

# fig.write_html("html/ESTADO_CLIENTE.html", full_html=False, include_plotlyjs="cdn")
# fig.write_image("imagens/ESTADO_CLIENTE.png", width=2677, height=450, scale=1)


- A maior parte dos clintes estão nos estados de **São Paulo, Goias e Distrito Federal**

---

## ANÁLISE EXPLORATÓRIA MAIS APROFUNDADA COM GRÁFICOS DE COLUNAS CATEGÓRICAS 

In [49]:
#fazendo o describe novamente das colunas categoricas
df_tratado.describe(exclude=["number"])

,CIDADE_CLIENTE,ESTADO_CLIENTE,FAIXA_IDADE_DATA_ASSINATURA_CONTRATO,FAIXA_VL_TOTAL_PC_PAGAS,FAIXA_VALOR_FINANCIAMENTO,FAIXA_VALOR_PARCELA,INADIMPLENTE_COBRANCA
count,10410,10410,10410,10410,10410,10410,10410
unique,525,27,7,4,4,4,2
top,BRASILIA,DF,25-34,de R$ 8.676 até R$ 25.000,Até R$ 210.000,Até R$ 2.500,SIM
freq,3734,4237,3288,2702,3411,3233,8035


In [50]:
#criando um dataframe com colunas categoricas apenas
df_categorico = df_tratado.select_dtypes(exclude=["number"])
df_categorico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   CIDADE_CLIENTE                        10410 non-null  category
 1   ESTADO_CLIENTE                        10410 non-null  category
 2   FAIXA_IDADE_DATA_ASSINATURA_CONTRATO  10410 non-null  category
 3   FAIXA_VL_TOTAL_PC_PAGAS               10410 non-null  category
 4   FAIXA_VALOR_FINANCIAMENTO             10410 non-null  category
 5   FAIXA_VALOR_PARCELA                   10410 non-null  category
 6   INADIMPLENTE_COBRANCA                 10410 non-null  category
dtypes: category(7)
memory usage: 185.2 KB


In [51]:
#mapeamento das cores
color_map = {
    "SIM": "#8B008B",
    "NAO": "#E6E6FA"
}

In [52]:
# #criacao de histogramas em comparacao com a coluna INADIMPLENTE_COBRANCA

# for column in df_categorico:
#     fig = px.histogram(df_tratado, x=column, color="INADIMPLENTE_COBRANCA", 
#                        color_discrete_map=color_map,
#                        text_auto=True,
#                        template="plotly_dark",
#                        barmode="group")
    
#     fig.update_layout(
#         bargap=0.2,
#         title_text=f"Histograma de {column} por INADIMPLENTE_COBRANCA", #titulo do grafico
#         title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#         title_x=0.5, #titulo centralizado
#         font=dict(size=14, color="darkgray"), #fonte
#         xaxis_title=column,
#         xaxis_title_font=dict(size=20, color="#cc7722"),
#         xaxis_tickformat=",.0f", #formatacao do eixo X
#         xaxis_type="category",
#         yaxis_title="Quantidade",
#         yaxis_tickformat=",.0f", #formatacao do eixo X #titulo do eixo Y
#         yaxis_showgrid=True, #mostrando as linhas no eixo Y
#         yaxis_gridwidth=1, #espessura das linhas no eixo Y
#         yaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo Y
#         template="plotly_dark",
#     )

#     fig.update_xaxes(categoryorder="category ascending")


#     fig.show()

#     fig.write_html(f"html/{column}_INADIMPLENTE_COBRANCA.html", full_html=False, include_plotlyjs="cdn")
#     fig.write_image(f"imagens/{column}_INADIMPLENTE_COBRANCA.png", width=2677, height=450, scale=1)

- Localidade

    * **Distribuição de valores**: A maioria dos clientes se encontra em **Brasília, Goiânia e São Paulo**.
    * **Inadimplência**: Existem poucos lugares onde o número de inadimplentes é menor que os não inadimplentes, normalmente em **pequenas cidades** com poucos casos disponíveis.

- Faixa Etária

    * **Distribuição de casos**: A maioria dos casos está na faixa de **25-54 anos** na data de assinatura do contrato.
    * **Inadimplência**: **Todas as faixas** apresentam mais inadimplentes que o contrário.

- Total de Parcelas Pagas

    * **Faixa Mais Comum**: Quando o total das parcelas pagas é maior que  **R$ 47.000**, vemos uma maior concentração de clientes.
    * **Tendência de Inadimplência**: Até **R$ 8.675** o número de inadimplentes é **menor** que os não inadimplentes, a medida que o valor **cresce**, **a proporção de inadimplentes cresce**.

- Valor de Financiamento

    * **Distribuição de Financiamento**: As faixas de valor de financiamento estão bem distribuidas, com uma maior concentração na menor faixa de **até R$ 210.000**.
    * **Comparação de Inadimplência**: Quanto **maior o financiamento**, mais **a proporção de inadimplentes cresce**.

- Valor da Parcela

    * **Distribuição da Parcela**: A faixa com uma maior concentração é de **até R$ 2.500**.
    * **Comparação de Inadimplência**: Os valores mais extremos de **até R$ 2.500** e de **mais de R$ 5.000** possuem uma **proporção menor** de não inadimplentes, sendo está **proporção maior** nas faixa do meio.

-  Número de Inadimplentes

    * **Inadimplência**: Cerca de **~80%** dos clientes está inadimplente.
---

## ANÁLISE EXPLORATÓRIA MAIS APROFUNDADA COM GRÁFICOS DE COLUNAS NUMÉRICAS

In [53]:
#construir um df apenas para as variaveis numericas

df_numerico = df_tratado.select_dtypes(include=["number"])
df_numerico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   RENDA_MENSAL_CLIENTE    10410 non-null  float32
 1   TAXA_AO_ANO             10410 non-null  float32
 2   PZ_FINANCIAMENTO        10410 non-null  int16  
 3   QT_PC_ATRASO            10410 non-null  int16  
 4   QT_DIAS_PRIM_PC_ATRASO  10410 non-null  int16  
 5   QT_TOTAL_PC_PAGAS       10410 non-null  int16  
 6   QT_PC_PAGA_EM_DIA       10410 non-null  int16  
 7   QT_DIAS_MIN_ATRASO      10410 non-null  int16  
 8   QT_DIAS_MEDIA_ATRASO    10410 non-null  int16  
 9   QT_DIAS_MAX_ATRASO      10410 non-null  int16  
dtypes: float32(2), int16(8)
memory usage: 325.3 KB


In [54]:
#criacao de histogramas em comparacao com a coluna INADIMPLENTE_COBRANCA

# for column in df_numerico:
#     fig = px.histogram(df_tratado, x=column, color="INADIMPLENTE_COBRANCA", 
#                        color_discrete_map=color_map,
#                        nbins=6, #aqui vamos agrupar os valores do eixo x, para melhorar a visibilidade dos graficos
#                        text_auto=True,
#                        template="plotly_dark",
#                        barmode="group")
    
#     fig.update_layout(
#         bargap=0.2,
#         title_text=f"Histograma de {column} por INADIMPLENTE_COBRANCA", #titulo do grafico
#         title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#         title_x=0.5, #titulo centralizado
#         font=dict(size=14, color="darkgray"), #fonte
#         xaxis_title=column,
#         xaxis_title_font=dict(size=20, color="#cc7722"),
#         xaxis_tickformat=",.0f", #formatacao do eixo X
#         yaxis_title="Quantidade",
#         yaxis_tickformat=",.0f", #formatacao do eixo X #titulo do eixo Y
#         yaxis_showgrid=True, #mostrando as linhas no eixo Y
#         yaxis_gridwidth=1, #espessura das linhas no eixo Y
#         yaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo Y
#         template="plotly_dark",
#     )

#     fig.update_xaxes(categoryorder="category ascending")


#     fig.show()

#     fig.write_html(f"html/{column}_INADIMPLENTE_COBRANCA.html", full_html=False, include_plotlyjs="cdn")
#     fig.write_image(f"imagens/{column}_INADIMPLENTE_COBRANCA.png", width=2677, height=450, scale=1)

- Renda Mensal

    * **Distribuição de Renda**: A maioria dos clientes se encontra em **até R$ 1.800** e **R$ 6.000 a R$ 8.000**.
    * **Taxas de Inadimplência**: Em todas as faixas de renda, o número de **inadimplentes** supera o de não inadimplentes.
    * **Tendência de Inadimplência**: Clientes com renda mensal de **até R$ 4.000** têm uma tendência significativamente maior de estar inadimplentes em comparação com aqueles em faixas de renda mais altas.

- Taxa Ao Ano

    * **Distribuição da Taxa Anual**: A maioria dos clientes paga uma taxa anual dentro da faixa de **15-20%**, tornando essa faixa a mais importante para análise.
    * **Taxas de Inadimplência por Taxa Anual**: Em todas as faixas de taxa anual, o número de **inadimplentes** supera o de não inadimplentes.

- Prazo de Financiamento

    * **Prazo de Financiamento Mais Comum**: O prazo de financiamento mais comum é **de 50-99 dias**.
    * **Tendência de Inadimplência**: Existe uma tendência onde, **a partir de 99 dias de financiamento, a diferença entre clientes inadimplentes e não inadimplentes aumenta**.

- Parcelas em Atraso

    * **Distribuição de Pagamentos em Atraso**: Em todos os gráficos relacionados a pagamentos em atraso, a concentração de clientes está no **primeiro intervalo** e **diminui gradualmente** à medida que os intervalos progridem.
    * **Comparação de Inadimplência**: Com algumas exceções, principalmente em **Dias do Primeiro Atraso**, o número de clientes inadimplentes é **sempre maior que o de não inadimplentes**.
---

In [55]:
# #verificando a correlacao das variaveis numericas

# fig = px.imshow(df_numerico.corr(), aspect="auto", text_auto=True, template="plotly_dark")
# fig.show()

# fig.write_html("html/Correlacao.html", full_html=False, include_plotlyjs="cdn")
# fig.write_image("imagens/Correlacao.png", width=2677, height=450, scale=1)

- Correlações

	* **Parcelas como Fator de Correlação**: A maioria das correlações fortes ocorre através das colunas que envolvem as **parcelas**.

	* **Prazo de Financiamento e Parcelas Pagas**: O Prazo de Financiamento apresenta uma correlação positiva média com:
		+ Total de Parcelas Pagas
		+ Parcelas Pagas em Dia

	* **Renda Mensal e Parcelas Pagas**: A Renda Mensal do cliente também apresenta uma correlação média com:
		+ Total de Parcelas Pagas
		+ Parcelas Pagas em Dia

	* **Taxa ao Ano e Prazo de Financiamento**: A Taxa ao Ano possuí um correlação média com:
		+ Prazo de Financiamento
		
	* **Correlação Negativa entre Atrasos e Pagamentos**: Existe uma correlação negativa forte entre:
		+ Quantidade de Parcelas em Atraso
		+ Quantidade de Dias da Primeira Parcela em Atraso
		e:
		+ Total de Parcelas Pagas
		+ Parcelas Pagas em Dia

---

## ONEHOTENCODING

In [56]:
#separando colunas numericas, coluna alvo e colunas categoricas
colunas_numericas = df_tratado.select_dtypes(include=["number"]).columns
coluna_alvo = df_tratado[["INADIMPLENTE_COBRANCA"]]

df_categorico = df_tratado.drop(columns=colunas_numericas).drop(columns=coluna_alvo)
df_categorico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   CIDADE_CLIENTE                        10410 non-null  category
 1   ESTADO_CLIENTE                        10410 non-null  category
 2   FAIXA_IDADE_DATA_ASSINATURA_CONTRATO  10410 non-null  category
 3   FAIXA_VL_TOTAL_PC_PAGAS               10410 non-null  category
 4   FAIXA_VALOR_FINANCIAMENTO             10410 non-null  category
 5   FAIXA_VALOR_PARCELA                   10410 non-null  category
dtypes: category(6)
memory usage: 175.0 KB


In [57]:
#vamos usar o Label Encoder para transformar nossas colunas categoricas em numerica para usar no modelo, pois temos muita cardinalidade na coluna de cidade e estado
lb = LabelEncoder()

for var in df_categorico:
    df_tratado[var] = lb.fit_transform(df_tratado[var])

df_tratado.sample(5)

,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,FAIXA_IDADE_DATA_ASSINATURA_CONTRATO,TAXA_AO_ANO,PZ_FINANCIAMENTO,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MEDIA_ATRASO,QT_DIAS_MAX_ATRASO,FAIXA_VL_TOTAL_PC_PAGAS,FAIXA_VALOR_FINANCIAMENTO,FAIXA_VALOR_PARCELA,INADIMPLENTE_COBRANCA
344,453,25,1800.00,3,20.50,220,0,0,30,14,1,3,17,1,1,0,SIM
1618,68,6,4800.00,2,17.00,180,0,0,105,42,1,101,4527,1,0,0,SIM
2173,301,8,5800.00,3,17.00,200,0,0,110,59,1,73,242,1,0,0,SIM
5596,453,25,1800.00,2,17.00,100,59,2873,14,8,2,7,28,3,1,1,NAO
2862,453,25,1800.00,3,17.00,100,0,0,39,35,6,11,20,1,1,1,SIM


In [58]:
df_tratado.shape

(10410, 17)

In [59]:
df_tratado.isnull().sum()

CIDADE_CLIENTE                          0
ESTADO_CLIENTE                          0
RENDA_MENSAL_CLIENTE                    0
FAIXA_IDADE_DATA_ASSINATURA_CONTRATO    0
TAXA_AO_ANO                             0
PZ_FINANCIAMENTO                        0
QT_PC_ATRASO                            0
QT_DIAS_PRIM_PC_ATRASO                  0
QT_TOTAL_PC_PAGAS                       0
QT_PC_PAGA_EM_DIA                       0
QT_DIAS_MIN_ATRASO                      0
QT_DIAS_MEDIA_ATRASO                    0
QT_DIAS_MAX_ATRASO                      0
FAIXA_VL_TOTAL_PC_PAGAS                 0
FAIXA_VALOR_FINANCIAMENTO               0
FAIXA_VALOR_PARCELA                     0
INADIMPLENTE_COBRANCA                   0
dtype: int64

In [60]:

df_tratado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   CIDADE_CLIENTE                        10410 non-null  int64   
 1   ESTADO_CLIENTE                        10410 non-null  int64   
 2   RENDA_MENSAL_CLIENTE                  10410 non-null  float32 
 3   FAIXA_IDADE_DATA_ASSINATURA_CONTRATO  10410 non-null  int64   
 4   TAXA_AO_ANO                           10410 non-null  float32 
 5   PZ_FINANCIAMENTO                      10410 non-null  int16   
 6   QT_PC_ATRASO                          10410 non-null  int16   
 7   QT_DIAS_PRIM_PC_ATRASO                10410 non-null  int16   
 8   QT_TOTAL_PC_PAGAS                     10410 non-null  int16   
 9   QT_PC_PAGA_EM_DIA                     10410 non-null  int16   
 10  QT_DIAS_MIN_ATRASO                    10410 non-null  int16   
 11  QT_DIAS

- Não temos valores **nulos**.
- Nossa única coluna não numérica agora é a **coluna alvo**.
-  Temos **10410 linhas** e **17 colunas** no nosso dataset.

---

In [61]:
#otimizando novamente as colunas
colunas_float = df_tratado.select_dtypes(include="float64").columns
colunas_int = df_tratado.select_dtypes(include="int64").columns
df_tratado[colunas_float] = df_tratado[colunas_float].apply(pd.to_numeric, downcast="float")
df_tratado[colunas_int] = df_tratado[colunas_int].apply(pd.to_numeric, downcast="integer")

df_tratado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   CIDADE_CLIENTE                        10410 non-null  int16   
 1   ESTADO_CLIENTE                        10410 non-null  int8    
 2   RENDA_MENSAL_CLIENTE                  10410 non-null  float32 
 3   FAIXA_IDADE_DATA_ASSINATURA_CONTRATO  10410 non-null  int8    
 4   TAXA_AO_ANO                           10410 non-null  float32 
 5   PZ_FINANCIAMENTO                      10410 non-null  int16   
 6   QT_PC_ATRASO                          10410 non-null  int16   
 7   QT_DIAS_PRIM_PC_ATRASO                10410 non-null  int16   
 8   QT_TOTAL_PC_PAGAS                     10410 non-null  int16   
 9   QT_PC_PAGA_EM_DIA                     10410 non-null  int16   
 10  QT_DIAS_MIN_ATRASO                    10410 non-null  int16   
 11  QT_DIAS

## SEPARAR A VARIAVEL TARGET DAS DEMAIS

In [62]:
#separando as variaveis preditoras da variavel alvo
var_preditoras = df_tratado.drop("INADIMPLENTE_COBRANCA", axis = 1)
print("Variáveis Preditoras: ",var_preditoras.shape)

var_target = df_tratado[["INADIMPLENTE_COBRANCA"]]
print("Variável Alvo: ", var_target.shape)

Variáveis Preditoras:  (10410, 16)
Variável Alvo:  (10410, 1)


In [63]:
#visualizando as variaveis preditoras
var_preditoras.sample(5)

,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,FAIXA_IDADE_DATA_ASSINATURA_CONTRATO,TAXA_AO_ANO,PZ_FINANCIAMENTO,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MEDIA_ATRASO,QT_DIAS_MAX_ATRASO,FAIXA_VL_TOTAL_PC_PAGAS,FAIXA_VALOR_FINANCIAMENTO,FAIXA_VALOR_PARCELA
8030,521,7,9800.00,3,17.00,72,0,0,55,55,0,0,0,1,3,3
814,163,8,7800.00,3,17.00,240,1,18,111,29,4,61,154,1,1,2
7002,163,8,7800.00,3,16.00,72,0,0,58,44,1,75,202,2,0,0
9519,453,25,1800.00,2,20.50,72,0,0,44,44,0,0,0,3,2,3
1843,512,10,4800.00,3,16.00,180,0,0,46,45,4,4,4,1,2,0


In [64]:
#visualizando a variavel alvo
var_target.sample(5)

,INADIMPLENTE_COBRANCA
3915,SIM
6509,SIM
8193,SIM
126,SIM
6780,SIM


## BALANCEAMENTO DA VARIÁVEL TARGET

In [65]:
#quantidade de dados da variavel alvo
df_tratado[["INADIMPLENTE_COBRANCA"]].value_counts()

INADIMPLENTE_COBRANCA
SIM                      8035
NAO                      2375
Name: count, dtype: int64

In [66]:
#balancear a variavel alvo para nao criar vies no modelo
balanceador = SMOTE()

var_preditoras_b, var_target_b = balanceador.fit_resample(var_preditoras, var_target)

In [67]:
#visualizando as variaveis preditoras balanceadas
var_preditoras_b.shape

(16070, 16)

In [68]:
var_preditoras_b.sample()

,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,FAIXA_IDADE_DATA_ASSINATURA_CONTRATO,TAXA_AO_ANO,PZ_FINANCIAMENTO,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MEDIA_ATRASO,QT_DIAS_MAX_ATRASO,FAIXA_VL_TOTAL_PC_PAGAS,FAIXA_VALOR_FINANCIAMENTO,FAIXA_VALOR_PARCELA
14561,68,6,7800.00,5,20.00,72,62,2278,1,1,0,0,0,0,1,2


In [69]:
#visualizando a variavel alvo balanceada
var_target_b.shape

(16070, 1)

In [70]:
# #grafico que mostra Inadimplentes x Não Inadimplentes
# fig = px.bar(var_target_b, color="value",
#              color_discrete_map=color_map)

# fig.update_traces(dict(marker_line_width=0))

# fig.update_layout(
#         title_text="Inadimplentes x Não Inadimplentes", #titulo do grafico
#         title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#         title_x=0.5, #titulo centralizado
#         font=dict(size=14, color="darkgray"), #fonte
#         xaxis_title="Inadimplência",
#         xaxis_title_font=dict(size=20, color="#cc7722"),
#         xaxis_tickformat=",.0f", #formatacao do eixo X
#         xaxis_type="category",
#         yaxis_title="Quantidade",
#         yaxis_tickformat=",.0f", #formatacao do eixo X #titulo do eixo Y
#         yaxis_showgrid=True, #mostrando as linhas no eixo Y
#         yaxis_gridwidth=1, #espessura das linhas no eixo Y
#         yaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo Y
#         coloraxis_colorbar=dict(x=1.1,title="Inadimplente"),
#         template="plotly_dark",
# )


# fig.show()

# fig.write_html("html/Balanceamento.html", full_html=False, include_plotlyjs="cdn")
# fig.write_image("imagens/Balanceamento.png", width=2677, height=450, scale=1)

- A variável alvo foi balanceada para não se criar um viés no modelo.

---

## ROBUSTSCALER E NORMALIZAÇÃO COM OS DADOS

In [71]:
#separando os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(var_preditoras_b, var_target_b, test_size=0.3)

In [72]:
print("Dados de Treino: ", X_train.shape)

Dados de Treino:  (11249, 16)


In [73]:
#RobustScaler, para dados com outliers
robusto = RobustScaler()
X_treino_robusto = robusto.fit_transform(X_train)
X_teste_robusto = robusto.transform(X_test)

In [74]:
#vendo o tamanho dos dados
X_treino_robusto.shape

(11249, 16)

In [75]:
#observando os dados
X_treino_robusto

array([[-0.16883117,  0.16666667, -1.        , ...,  0.        ,
         1.        , -1.        ],
       [ 0.8       ,  1.4166666 ,  0.33333334, ..., -0.5       ,
         1.        ,  0.5       ],
       [ 0.04675325,  0.        , -0.33333334, ..., -0.5       ,
        -0.5       , -1.        ],
       ...,
       [-0.35324675,  0.        ,  0.        , ..., -0.5       ,
        -0.5       ,  0.        ],
       [ 0.04675325,  0.        , -0.33333334, ..., -0.5       ,
        -0.5       , -1.        ],
       [-0.2       , -0.16666667,  0.        , ...,  1.        ,
         0.        , -0.5       ]], dtype=float32)

In [76]:
#normalizacao
normalizador = MinMaxScaler()
X_treino_normalizado = normalizador.fit_transform(X_train)
X_teste_normalizado = normalizador.transform(X_test)

In [77]:
#vendo o tamanho dos dados
X_treino_normalizado.shape

(11249, 16)

In [78]:
#observando os dados
X_treino_normalizado

array([[0.15267175, 0.3846154 , 0.        , ..., 0.33333334, 1.        ,
        0.        ],
       [0.8645038 , 0.9615385 , 1.        , ..., 0.        , 1.        ,
        1.        ],
       [0.3110687 , 0.30769232, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01717557, 0.30769232, 0.75      , ..., 0.        , 0.        ,
        0.6666667 ],
       [0.3110687 , 0.30769232, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.129771  , 0.23076925, 0.75      , ..., 1.        , 0.33333334,
        0.33333334]], dtype=float32)

## Modelos de Machine Learning

In [79]:
# #usando o RandomForest
# rf = RandomForestClassifier()

# # Valores para o grid de hiperparametros
# n_estimators = np.array([100,200,300,400,500])
# max_depth = np.array([10,20,30])
# criterion = np.array(["gini", "entropy"])
# max_features = np.array(["sqrt", "log2", None])
# min_samples_split = np.array([1,2,5])
# min_samples_leaf = np.array([1,2,3])

# # Grid de hiperparâmetros
# grid_parametros = dict(n_estimators = n_estimators,
#                     max_depth = max_depth,
#                     criterion = criterion,
#                     max_features = max_features,
#                     min_samples_split = min_samples_split,
#                     min_samples_leaf = min_samples_leaf)


# # Criando o modelo com o Grid de Hiperparametros
# randomForest = GridSearchCV(rf, grid_parametros, cv = 3, n_jobs = 8)

In [80]:
# # Treinando o modelo normalizado
# inicio = time.time()
# randomForest.fit(X_treino_normalizado, y_train)
# fim = time.time()

In [81]:
# # Obtendo e visualizando os parametros treinados
# treino_rf_n = pd.DataFrame(randomForest.cv_results_)

In [82]:
# # Acurácia em Treino normalizado
# print(f"Acurácia em Treinamento: {randomForest.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {randomForest.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo (s): ", round(fim - inicio,2))
# print("")
# print("Número de treinamentos realizados: ", treino_rf_n.shape[0])

In [83]:
# #Treinando o modelo robusto
# inicio = time.time()
# randomForest.fit(X_treino_robusto, y_train)
# fim = time.time()

In [84]:
# treino_rf_r = pd.DataFrame(randomForest.cv_results_)

In [85]:
# # Acurácia em Treino normalizado
# print(f"Acurácia em Treinamento: {randomForest.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {randomForest.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo (s): ", round(fim - inicio,2))
# print("")
# print("Número de treinamentos realizados: ", treino_rf_r.shape[0])

In [86]:
# #Dicionário de métricas e metadados para comparar com outros modelos
# modelo_rf = {"Melhores Hiperparâmetros":randomForest.best_params_,
#             "Número de Modelos Treinados": treino_rf_n.shape[0],
#             "Melhor Score": str(round(randomForest.best_score_ * 100,2))+"%"}

# display(modelo_rf)

- No **Random Forest Classifier**, tanto os dados **normalizados** ou usando o **Robust Scaler** obterem resultados semelhantes, portanto, iremos usar apenas a **normalização** nos próximos modelos.

- **Acurácia** em Treinamento: **90.81%**

- **Hiperparâmetros Ideais**: {'criterion': np.str_('entropy'), 'max_depth': np.int64(30), 'max_features': 'log2', 'min_samples_leaf': np.int64(1), 'min_samples_split': np.int64(2), 'n_estimators': np.int64(200)}

- Tempo de Treinamento do Modelo:  **647.77 segundos**

- Número de treinamentos realizados:  **810**

---

In [87]:
# #testando o modelo SVM
# svm = SVC(probability=False)

# # Valores para o grid de hiperparametros
# grid_parametros = {"C": [0.1,1,10,100,1000],
#                     "gamma": [1,0.1,0.01,0.001],
#                     #"kernel": ["poly","rbf","sigmoid","linear"], -> demoram bastante
#                     "kernel": ["linear"], # -> mais rapido
#                     "degree" : [2,3,4,],
#                     "coef0" : [0.5,1],
#                     #"decision_function_shape":["ovo", "ovr"], -> demoram bastante
#                     "max_iter": [-1, 1]}


# supportvectorm = GridSearchCV(svm, grid_parametros, n_jobs = 8)

In [88]:
# # Treinando os modelos com os dados normalizados
# inicio = time.time()
# supportvectorm.fit(X_treino_normalizado, y_train)
# fim = time.time()

In [89]:
# # Obtendo e visualizando os parametros treinados
# treino_svm_n = pd.DataFrame(supportvectorm.cv_results_)

In [90]:
# # Acurácia em Treino
# print(f"Acurácia em Treinamento: {supportvectorm.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {supportvectorm.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo (s): ", round(fim - inicio,2))
# print("")
# print("Número de treinamentos realizados: ", treino_svm_n.shape[0])

In [91]:
# # Dicionário de métricas e metadados para comparar com outros modelos
# modelo_svm = {"Melhores Hiperparâmetros":supportvectorm.best_params_,
#             "Número de Modelos Treinados": treino_svm_n.shape[0],
#             "Melhor Score": str(round(supportvectorm.best_score_ * 100,2))+"%"}

# display(modelo_svm)

- O modelo **SVM** apresentou uma acurácia pior que o **Random Forest**, mesmo mudando alguns parâmetros.

- **Acurácia** em Treinamento: **83.02%**

- **Hiperparâmetros** Ideais: {'C': 10, 'coef0': 0.5, 'degree': 2, 'gamma': 1, 'kernel': 'linear', 'max_iter': -1}

- Tempo de Treinamento do Modelo:  **228.19 segundos**

- Número de treinamentos realizados: **240**

---

In [92]:
# #testando o modelo KNN
# knn = KNeighborsClassifier()

# # Valores para o grid de hiperparametros
# n_neighbors = np.array([3,5,7,9])
# algorithm = np.array(["auto", "ball_tree", "kd_tree", "brute"])
# leaf_size = np.array([30,31,32])
# metric =  np.array(["minkowski","euclidean"])


# # Grid de hiperparâmetros
# grid_parametros = dict(n_neighbors = n_neighbors,
#                     algorithm = algorithm,
#                     leaf_size = leaf_size,
#                     metric = metric)

# knn = GridSearchCV(knn, grid_parametros, n_jobs = 8)

In [93]:
# # Treinando os modelos
# inicio = time.time()
# knn.fit(X_treino_normalizado, y_train)
# fim = time.time()

In [94]:
# # Obtendo e visualizando os parametros treinados
# treino_knn = pd.DataFrame(knn.cv_results_)

In [95]:
# # Acurácia em Treino
# print(f"Acurácia em Treinamento: {knn.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {knn.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo (s): ", round(fim - inicio,2))
# print("")
# print("Número de treinamentos realizados: ", treino_knn.shape[0])

In [96]:
# # Dicionário de métricas e metadados para comparar com outros modelos
# modelo_knn = {"Melhores Hiperparâmetros":knn.best_params_,
#             "Número de Modelos Treinados": treino_knn.shape[0],
#             "Melhor Score": str(round(knn.best_score_ * 100,2))+"%"}

# display(modelo_knn)

- O modelo **KNN** também apresentou uma acurácia pior que o **Random Forest**, e foi melhor que o **SVM**.

- **Acurácia** em Treinamento: **86.47%**

- **Hiperparâmetros** Ideais: {'algorithm': np.str_('auto'), 'leaf_size': np.int64(30), 'metric': np.str_('minkowski'), 'n_neighbors': np.int64(3)}

- Tempo de Treinamento do Modelo:  **9.37 segundos**

- Número de treinamentos realizados: **96**

---

In [97]:
# usando o label encoder novamente para o XGBoost, pois o y_train e o y_test precisam ser transformados para usar o XGBoost
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [98]:
# #testando o modelo XGBoost
# xgb = XGBClassifier()

# # Valores para o grid de hiperparametros
# booster  = np.array(["gbtree","gblinear"])
# silent  = np.array([0])
# min_child_weight = np.array([1,2,3])
# gamma = np.array([0,0.1,0.2])
# max_depth = np.array([3,5,7,10,20,50])
# colsample_bytree = np.array([0.5, 0.6, 0.7, 0.8, 0.9])


# # Grid de hiperparâmetros
# grid_parametros = dict(booster = booster,
#                     silent = silent,
#                     min_child_weight = min_child_weight,
#                     gamma = gamma,
#                     max_depth = max_depth,
#                     colsample_bytree = colsample_bytree)

# xgb = GridSearchCV(xgb, grid_parametros, n_jobs = 8)

In [99]:
# # Treinando os modelos
# inicio = time.time()
# xgb.fit(X_treino_normalizado, y_train)
# fim = time.time()

In [100]:
# # Obtendo e visualizando os parametros treinados
# treino_xgb = pd.DataFrame(xgb.cv_results_)

In [101]:
# # Acurácia em Treino
# print(f"Acurácia em Treinamento: {xgb.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {xgb.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo (s): ", round(fim - inicio,2))
# print("")
# print("Número de treinamentos realizados: ", treino_xgb.shape[0])

In [102]:
# # Dicionário de métricas e metadados para comparar com outros modelos
# modelo_xgb = {"Melhores Hiperparâmetros":xgb.best_params_,
#             "Número de Modelos Treinados": treino_xgb.shape[0],
#             "Melhor Score": str(round(xgb.best_score_ * 100,2))+"%"}

# display(modelo_xgb)

- O modelo **XGBoost** teve o melhor desempenho até agora entre os modelos.

- **Acurácia** em Treinamento: **91.69%**

- **Hiperparâmetros** Ideais: {'booster': np.str_('gbtree'), 'colsample_bytree': np.float64(0.8), 'gamma': np.float64(0.0), 'max_depth': np.int64(50), 'min_child_weight': np.int64(2), 'silent': np.int64(0)}

- Tempo de Treinamento do Modelo:  **35.79 segundos**

- Número de treinamentos realizados: **540**

---

In [103]:
# #regressao logistica

# rl = LogisticRegression()

# # Valores para o grid de hiperparametros
# max_iter = np.array([10,100,1000,10000])
# C = np.array([0.01, 0.1, 1, 10, 100])
# penalty = np.array(["l1", "l2"])
# solver =  np.array(["liblinear","saga"])


# # Grid de hiperparâmetros
# grid_parametros = dict(max_iter = max_iter,
#                     C = C,
#                     penalty = penalty,
#                     solver = solver)

# rl = GridSearchCV(rl, grid_parametros, n_jobs = 8)

In [104]:
# # Treinando os modelos
# inicio = time.time()
# rl.fit(X_treino_normalizado, y_train)
# fim = time.time()

In [105]:
# # Obtendo e visualizando os parametros treinados
# treino_rl = pd.DataFrame(rl.cv_results_)

In [106]:
# # Acurácia em Treino
# print(f"Acurácia em Treinamento: {rl.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {rl.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo (s): ", round(fim - inicio,2))
# print("")
# print("Número de treinamentos realizados: ", treino_rl.shape[0])

In [107]:
# # # Dicionário de métricas e metadados para comparar com outros modelos
# modelo_rl = {"Melhores Hiperparâmetros":rl.best_params_,
#             "Número de Modelos Treinados": treino_rl.shape[0],
#             "Melhor Score": str(round(rl.best_score_ * 100,2))+"%"}

# display(modelo_rl)

- O modelo **Regressão Linear** apresentou uma acurácia pior que o **XGBoost**.

- **Acurácia** em Treinamento: **83.16%**

- **Hiperparâmetros** Ideais: {'C': np.float64(0.1), 'max_iter': np.int64(10), 'penalty': np.str_('l1'), 'solver': np.str_('liblinear')}

- Tempo de Treinamento do Modelo:  **3.53 segundos**

- Número de treinamentos realizados: **80**

---

- Vamos usar o modelo **XGBoost**, pois teve o melhor desempenho entre todos os testados.
---

In [108]:
# Criando o modelo com XGBoost
xg = XGBClassifier(booster  = "gbtree", silent = 0, min_child_weight = 2, 
                            gamma = 0.0, max_depth = 50, colsample_bytree = 0.8, n_jobs = 8)


# Construção do modelo
xg = xg.fit(X_treino_normalizado, y_train)

y_pred = xg.predict(X_teste_normalizado) 

#metricas
accuracy = accuracy_score(y_test, y_pred) 
print(f"Accuracy: {accuracy * 100:.2f}%")

precision = precision_score(y_test, y_pred, average="binary") 
print(f"Precision: {precision:.2f}")

recall = recall_score(y_test, y_pred, average="binary")
print(f"Recall: {recall:.2f}")

f1 = f1_score(y_test, y_pred, average="binary")
print(f"F1 Score: {f1:.2f}")

conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{conf_matrix}")

roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC Score: {roc_auc:.2f}")

class_report = classification_report(y_test, y_pred)
print(f"Classification Report:\n{class_report}")


Accuracy: 91.18%
Precision: 0.91
Recall: 0.91
F1 Score: 0.91
Confusion Matrix:
[[2194  210]
 [ 215 2202]]
ROC AUC Score: 0.91
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.91      0.91      2404
           1       0.91      0.91      0.91      2417

    accuracy                           0.91      4821
   macro avg       0.91      0.91      0.91      4821
weighted avg       0.91      0.91      0.91      4821



- **Acurácia: 92.62%:** Aproximadamente **92%** das previsões feitas pelo modelo estão corretas.

- **Precisão: 0.93:** De todas as previsões que o modelo identificou como positivas,**92%** delas eram realmente positivas. Uma **alta precisão** indica uma **baixa taxa** de falsos positivos.

- **Recall: 0.92:** De todos os casos positivos reais, o modelo identificou corretamente **92%** deles. Um alto recall significa uma **baixa taxa** de falsos negativos.

- **F1 Score: 0.92:** A média harmônica da precisão e do recall. Um alto F1 Score indica um **bom equilíbrio** entre precisão e recall.

- **Matriz de Confusão:** 
    - **Verdadeiros Negativos (2241):** Casos negativos corretamente identificados.
    - **Falsos Positivos (189):** Casos positivos incorretamente identificados.
    - **Falsos Negativos (197):** Casos negativos incorretamente identificados.
    - **Verdadeiros Positivos (2194):** Casos positivos corretamente identificados. Isso mostra que tanto os falsos positivos quanto os falsos negativos são **relativamente baixos**, indicando um **bom desempenho**.
    
- **ROC AUC Score: 0.92:** Área sob a curva ROC. Um score de **0.92** indica que o modelo faz um **bom trabalho** em distinguir entre classes positivas e negativas.

- **Classificação:**
    - **Classe 0 e Classe 1:** Ambas as classes têm alta precisão, recall e F1 scores, mostrando desempenho equilibrado em diferentes categorias.
    - **Média Macro:** Média da precisão, recall e F1 scores igualmente entre as classes.
    - **Média Ponderada:** Leva em conta o desbalanceamento das classes ao fazer a média da precisão, recall e F1 scores.
---

In [109]:
# #vendo a importancia das variaveis no modelo
importancia = pd.Series(data=xg.feature_importances_, index=var_preditoras_b.columns)
importancia = importancia.sort_values(ascending=True)

# fig = px.bar(x=importancia, y=importancia.index, orientation="h", 
#             title="Importância das colunas para o modelo",
#             color = importancia,
#             color_continuous_scale = "Purples")

# fig.update_layout(
#     title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#     title_x=0.5, #titulo centralizado
#     font=dict(size=14, color="darkgray"), #fonte
#     xaxis_title="Importância",
#     xaxis_title_font=dict(size=20, color="#cc7722"),
#     yaxis_title="Variável",
#     yaxis_title_font=dict(size=20, color="#cc7722"),
#     yaxis=dict(tickfont=dict(size=12)),
#     yaxis_showgrid=True, #mostrando as linhas no eixo Y
#     yaxis_gridwidth=1, #espessura das linhas no eixo Y  
#     coloraxis_colorbar=dict(x=1.1,title="Importância"),         
#     template="plotly_dark"

# )

# fig.show()

# fig.write_html("html/Importancia_Variaveis.html", full_html=False, include_plotlyjs="cdn") #convertendo para html
# pio.write_image(fig, "imagens/Importancia_Variaveis.png", width=2677, height=450 , scale=1) #convertendo para png

In [110]:
# Visualizando o percentual de importancia de cada variável
importancia.sort_values(ascending = False)

PZ_FINANCIAMENTO                       0.18
QT_PC_ATRASO                           0.16
QT_DIAS_PRIM_PC_ATRASO                 0.14
RENDA_MENSAL_CLIENTE                   0.13
FAIXA_VALOR_PARCELA                    0.05
TAXA_AO_ANO                            0.05
FAIXA_VALOR_FINANCIAMENTO              0.05
FAIXA_VL_TOTAL_PC_PAGAS                0.03
QT_DIAS_MIN_ATRASO                     0.03
QT_TOTAL_PC_PAGAS                      0.03
ESTADO_CLIENTE                         0.03
QT_PC_PAGA_EM_DIA                      0.03
QT_DIAS_MAX_ATRASO                     0.02
QT_DIAS_MEDIA_ATRASO                   0.02
CIDADE_CLIENTE                         0.02
FAIXA_IDADE_DATA_ASSINATURA_CONTRATO   0.02
dtype: float32

- As colunas mais importantes para o modelo são: **PZ_FINANCIAMENTO**, **QT_PC_ATRASO**, **RENDA_MENSAL_CLIENTE**, **QT_DIAS_PRIM_PC_ATRASO**
---

In [111]:
#simulando os dados de treino
resultado = xg.score(X_treino_normalizado, y_train)
print(resultado)

0.9991110320917415


In [112]:
#simulando os dados de teste (valor sempre menor do que os dados de treino)
resultado = xg.score(X_teste_normalizado, y_test)
print(resultado)

0.9118440157643642


In [113]:
# Gerando o DataFrame com todos os valores de todos os modelos treinados
# resumo = pd.DataFrame({"Random Forest":pd.Series(modelo_rf),
#                     "SVM":pd.Series(modelo_svm),
#                     "Linear Regression": pd.Series(modelo_rl),
#                     "XGBoost": pd.Series(modelo_xgb),
#                     "KNN":pd.Series(modelo_knn)})

In [114]:
#resumo

In [115]:
#salvando o modelo
joblib.dump(xg, "modelo/modelo.pk")

['modelo/modelo.pk']

## ANÁLISE PRESCRITIVA

Algumas medidas podem ser tomadas para minimizar o número de clientes inadimplentes:

1. **Segmentação de clientes**

    - Utilizando algoritmos de agrupamento, para agrupar clientes com base em fatores de risco (ex.: renda, histórico de crédito, comportamento de pagamento), pode ajudar a personalizar intervenções, como oferecer planos de pagamento flexíveis ou taxas de juros ajustadas para grupos de alto risco.
    - Para estes clientes, seria interessante construir um sistema de alerta que seja ativado quando os mesmos mostram sinais iniciais de dificuldades financeiras, como atrasos em pagamentos.

2. **Análise de Comportamento e Temporal**

    - Outra alternativa seria a análise de históricos de pagamento ao longo do tempo para detectar tendências. Identificar esses padrões pode ajudar a planejar intervenções, como lembretes direcionados durante os períodos de maior risco.

3. **Ajustes de Política com Base em Perfis de Risco**

    - Para contratos com previsão de alta probabilidade de inadimplência, poderiam ser definidas políticas como taxas de financiamento menores, prazos de empréstimo mais curtos ou um maior rigor no processo de aprovação.
    - Ofereçer preços baseados nos diferentes grupos de clientes, ajustando as taxas de juros com base na probabilidade de inadimplência.

4. **Programas de Educação e Engajamento do Cliente**

    - Programas para educar clientes sobre gestão financeira e orçamento podem ser uma boa alternativa, especialmente para aqueles em segmentos de alto risco. Isso pode reduzir o risco de inadimplência ao melhorar a tomada de decisões financeiras.
    - Lembretes e dicas por meio de notificações ou e-mails para incentivar pagamentos pontuais.


## Conclusão

Neste projeto, foi desenvolvida uma análise para **prever a probabilidade de inadimplência de clientes** com base em um conjunto de dados de empréstimos. As principais etapas do projeto incluíram:

1. **Engenharia de Atributos e Preparação dos Dados**: Realizamos um **pré-processamento** completo dos dados, incluindo a criação de novas variáveis e o **balanceamento da variável alvo** para garantir uma melhor performance dos modelos.

2. **Análise Exploratória de Dados (EDA)**: Utilizamos visualizações e estatísticas descritivas para **identificar padrões e entender a distribuição das variáveis**, o que ajudou a guiar as etapas subsequentes de modelagem.

3. **Modelagem**: Foram testados diferentes **algoritmos de classificação**, e os resultados de cada modelo foram avaliados devido sua **acurácia**. O modelo que apresentou o melhor desempenho foi o **XGBoost**, alcançando uma acurácia de aproximadamente **92%** e um F1-score de **0.92**.

4. **Interpretabilidade**: Utilizamos valores **SHAP** para interpretar as previsões do modelo final, identificando as **variáveis mais influentes** para a probabilidade de inadimplência (**PZ_FINANCIAMENTO**, **QT_PC_ATRASO**, **RENDA_MENSAL_CLIENTE**, **QT_DIAS_PRIM_PC_ATRASO**).

O modelo final conseguiu identificar com precisão os clientes mais propensos a inadimplência, destacando variáveis-chave como **PZ_FINANCIAMENTO**, **QT_PC_ATRASO**, **RENDA_MENSAL_CLIENTE**, **QT_DIAS_PRIM_PC_ATRASO**. Este modelo pode ser aplicado para ajudar instituições financeiras a reduzir o risco, identificando possíveis inadimplentes e permitindo ações preventivas.

Em conclusão, o projeto forneceu uma solução eficiente para previsão de inadimplência, com alto potencial de aplicação prática para melhorar a gestão de risco e tomada de decisões no setor financeiro.

